In [11]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
# For time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

# Correct way
base_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
multi_output_regressor = MultiOutputRegressor(base_regressor)  # ✅ Pass configured estimator

# For machine learning
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans
from xgboost import XGBRegressor
import lightgbm as lgb


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# For time series analysis and models
from statsmodels.tsa.seasonal import seasonal_decompose
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

# For machine learning pipelines and evaluation
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer

import logging
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s: %(message)s')

In [12]:
# Load the preprocessed data
df = pd.read_csv('data/preprocessed_tollplaza_data.csv')
df['initiated_time'] = pd.to_datetime(df['initiated_time'])
df['time_interval'] = pd.to_datetime(df['time_interval'])
print(df['initiated_time'])
print("Data loaded successfully. Shape:", df.shape)

0        2024-03-19 21:54:00
1        2024-03-19 18:59:00
2        2024-03-19 00:14:00
3        2024-03-19 18:19:00
4        2024-03-19 15:03:00
                 ...        
367069   2024-03-19 07:31:00
367070   2024-03-19 12:55:00
367071   2024-03-19 13:11:00
367072   2024-03-19 17:19:00
367073   2024-03-19 17:28:00
Name: initiated_time, Length: 367074, dtype: datetime64[ns]
Data loaded successfully. Shape: (367074, 40)


In [13]:

# 1. TRAFFIC PATTERN ANALYSIS
print("\n1. TRAFFIC PATTERN ANALYSIS")
print("-" * 50)

# Aggregate data by time intervals
def create_time_series_data(df, interval='15min'):
    """Create time series data at specified intervals."""
    ts_data = df.set_index('initiated_time').resample(interval).agg({
        'txn_amount': 'sum',
        'SlNo.': 'count',  # Count of transactions
        'inn_rr_time_sec': 'mean'  # Average processing time
    }).reset_index()
    
    ts_data.rename(columns={'SlNo.': 'transaction_count'}, inplace=True)
    return ts_data

# Create 15-minute interval data
ts_15min = create_time_series_data(df, '15min')
print("Time series data created at 15-minute intervals:")
print(ts_15min.head())

# Create hourly data
ts_hourly = create_time_series_data(df, '1H')
print("\nTime series data created at hourly intervals:")
print(ts_hourly.head())

# Visualize traffic patterns
plt.figure(figsize=(14, 7))
plt.plot(ts_15min['initiated_time'], ts_15min['transaction_count'])
plt.title('Traffic Volume Over Time (15-minute intervals)')
plt.xlabel('Time')
plt.ylabel('Number of Transactions')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('images/traffic_pattern_15min.png')
plt.close()

# Decompose time series to analyze trend, seasonality, and residuals
def analyze_time_series_components(ts_data, column='transaction_count'):
    """Decompose time series into trend, seasonality, and residual components."""
    # Set the index to datetime for decomposition
    ts = ts_data.set_index('initiated_time')[column]
    
    # Decompose the time series
    decomposition = seasonal_decompose(ts, model='additive', period=12)  # 24 periods for hourly data
    
    # Plot the decomposition
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(14, 12))
    decomposition.observed.plot(ax=ax1)
    ax1.set_title('Observed')
    decomposition.trend.plot(ax=ax2)
    ax2.set_title('Trend')
    decomposition.seasonal.plot(ax=ax3)
    ax3.set_title('Seasonality')
    decomposition.resid.plot(ax=ax4)
    ax4.set_title('Residuals')
    plt.tight_layout()
    plt.savefig('images/time_series_decomposition.png')
    plt.close()
    
    return decomposition

# Analyze hourly traffic patterns
print("\nAnalyzing hourly traffic patterns...")
decomposition = analyze_time_series_components(ts_hourly)



1. TRAFFIC PATTERN ANALYSIS
--------------------------------------------------
Time series data created at 15-minute intervals:
       initiated_time  txn_amount  transaction_count  inn_rr_time_sec
0 2024-03-19 00:00:00      266256               2469      1882.420008
1 2024-03-19 00:15:00      281348               2518       958.048848
2 2024-03-19 00:30:00      233571               2091       486.887135
3 2024-03-19 00:45:00      237070               2015      1060.671464
4 2024-03-19 01:00:00      245842               2097      2214.508822

Time series data created at hourly intervals:
       initiated_time  txn_amount  transaction_count  inn_rr_time_sec
0 2024-03-19 00:00:00     1018245               9093      1123.435170
1 2024-03-19 01:00:00      917015               7692      1478.759880
2 2024-03-19 02:00:00      790608               6569       913.394885
3 2024-03-19 03:00:00      755165               6123       417.210681
4 2024-03-19 04:00:00      805013               6792  

In [14]:
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Import regressors
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.multioutput import MultiOutputRegressor

# -------------------------------
# 2. IMPROVED PREDICTIVE MODELS DEVELOPMENT
# -------------------------------
logging.info("2. IMPROVED PREDICTIVE MODELS DEVELOPMENT")

# 2.1 Traffic Volume Prediction Model
logging.info("2.1 Traffic Volume Prediction Model")

# -------------------------------
# 2.1.1 Enhanced Feature Preparation
# -------------------------------
def prepare_features_for_prediction(df):
    """
    Prepare features for traffic prediction models with enhanced feature engineering.
    Adds time-based features, cyclical encodings, multiple lag features, rolling statistics,
    transaction differences, transaction rates, average amounts per transaction,
    and improved merchant/vehicle class features.
    """
    features = df.copy()
    
    # Time-based features
    features['hour'] = features['initiated_time'].dt.hour
    features['day_of_week'] = features['initiated_time'].dt.dayofweek
    features['is_weekend'] = features['day_of_week'].isin([5, 6]).astype(int)
    features['quarter_of_day'] = features['hour'] // 6
    features['day_part'] = pd.cut(features['hour'], 
                                  bins=[0, 6, 12, 18, 24], 
                                  labels=['night', 'morning', 'afternoon', 'evening'])
    features['day_part'] = features['day_part'].astype('category').cat.codes
    
    # Create cyclical time features
    features['hour_sin'] = np.sin(2 * np.pi * features['hour'] / 24)
    features['hour_cos'] = np.cos(2 * np.pi * features['hour'] / 24)
    features['day_sin'] = np.sin(2 * np.pi * features['day_of_week'] / 7)
    features['day_cos'] = np.cos(2 * np.pi * features['day_of_week'] / 7)
    
    # Add lag features with more historical context
    for lag in range(1, 6):  # Use 5 lag periods
        features[f'lag_{lag}'] = features['transaction_count'].shift(lag)
    
    # Add rolling statistics
    features['rolling_mean_3'] = features['transaction_count'].rolling(window=3).mean()
    features['rolling_std_3'] = features['transaction_count'].rolling(window=3).std()
    features['rolling_mean_6'] = features['transaction_count'].rolling(window=6).mean()
    features['rolling_max_6'] = features['transaction_count'].rolling(window=6).max()
    features['rolling_min_6'] = features['transaction_count'].rolling(window=6).min()
    
    # Calculate transaction differences
    features['tx_diff_1'] = features['transaction_count'].diff(1)
    features['tx_diff_2'] = features['transaction_count'].diff(2)
    
    # Add transaction rate features
    if 'inn_rr_time_sec' in features.columns:
        features['tx_per_second'] = features['transaction_count'] / features['inn_rr_time_sec']
        features['tx_per_second'].fillna(features['tx_per_second'].mean(), inplace=True)
    
    # Add average transaction amount per transaction
    if 'txn_amount' in features.columns:
        features['avg_amount_per_tx'] = features['txn_amount'] / features['transaction_count']
        features['avg_amount_per_tx'].fillna(features['avg_amount_per_tx'].mean(), inplace=True)
    
    # IMPROVED: Enhanced merchant_name features for traffic prediction
    if 'merchant_name' in features.columns:
        # Get the top N merchants by transaction volume (increased from 5 to 10)
        top_merchants = features['merchant_name'].value_counts().head(10).index.tolist()
        
        # Create time-based merchant distribution
        merchant_dummies = pd.get_dummies(features['merchant_name'], prefix='merchant')
        merchant_dummies['initiated_time'] = features['initiated_time']
        
        # Group by the time interval used for your main data
        merchant_agg = merchant_dummies.groupby(features['initiated_time']).sum()
        
        # Calculate both absolute counts and percentage distribution
        # First add absolute counts
        features = features.join(merchant_agg, on='initiated_time', how='left')
        
        # Then calculate percentage distribution
        merchant_total = merchant_agg.sum(axis=1)
        merchant_pct = merchant_agg.div(merchant_total, axis=0) * 100
        
        # Add percentage columns with _pct suffix
        for m in top_merchants:
            merchant_col = f'merchant_{m}'
            if merchant_col in merchant_pct.columns:
                features[f'{merchant_col}_pct'] = merchant_pct[merchant_col]
        
        # Also add a feature for "other" merchants (combined)
        top_merchant_cols = [f'merchant_{m}' for m in top_merchants if f'merchant_{m}' in merchant_agg.columns]
        if top_merchant_cols:
            features['other_merchants'] = merchant_total - merchant_agg[top_merchant_cols].sum(axis=1)
            features['other_merchants_pct'] = 100 - merchant_pct[top_merchant_cols].sum(axis=1)
        
        # Fill any NaNs with appropriate values (0 for counts, mean for percentages)
        for col in features.columns:
            if col.startswith('merchant_'):
                if col.endswith('_pct'):
                    features[col].fillna(features[col].mean(), inplace=True)
                else:
                    features[col].fillna(0, inplace=True)
    
    # IMPROVED: Enhanced vehicle_class_code features for traffic prediction
    if 'vehicle_class_code' in features.columns:
        # Get the top N vehicle classes by frequency (increased from 5 to all classes)
        all_vehicle_classes = features['vehicle_class_code'].unique()
        
        # Create time-based vehicle class distribution
        vc_dummies = pd.get_dummies(features['vehicle_class_code'], prefix='vc')
        vc_dummies['initiated_time'] = features['initiated_time']
        
        # Group by the same time interval as main data
        vc_agg = vc_dummies.groupby(features['initiated_time']).sum()
        
        # Add absolute counts first
        features = features.join(vc_agg, on='initiated_time', how='left')
        
        # Calculate percentage distribution for each vehicle class
        vc_total = vc_agg.sum(axis=1)
        vc_pct = vc_agg.div(vc_total, axis=0) * 100
        
        # Add percentage columns with _pct suffix
        for vc in all_vehicle_classes:
            vc_col = f'vc_{vc}'
            if vc_col in vc_pct.columns:
                features[f'{vc_col}_pct'] = vc_pct[vc_col]
        
        # Also add summary features:
        # 1. Ratio of heavy to light vehicles (if you have a way to categorize them)
        # This depends on your vehicle class coding system, modify as needed
        heavy_vehicle_codes = [vc for vc in all_vehicle_classes if isinstance(vc, (int, str)) and str(vc).isdigit() and int(str(vc)[0]) > 3]
        if heavy_vehicle_codes:
            heavy_cols = [f'vc_{vc}' for vc in heavy_vehicle_codes if f'vc_{vc}' in vc_agg.columns]
            light_cols = [col for col in vc_agg.columns if col not in heavy_cols]
            
            if heavy_cols and light_cols:
                features['heavy_vehicle_count'] = vc_agg[heavy_cols].sum(axis=1)
                features['light_vehicle_count'] = vc_agg[light_cols].sum(axis=1)
                features['heavy_to_light_ratio'] = features['heavy_vehicle_count'] / features['light_vehicle_count']
                features['heavy_vehicle_pct'] = features['heavy_vehicle_count'] / vc_total * 100
        
        # Fill NaNs appropriately
        for col in features.columns:
            if col.startswith('vc_'):
                if col.endswith('_pct'):
                    features[col].fillna(features[col].mean(), inplace=True)
                else:
                    features[col].fillna(0, inplace=True)
            elif col in ['heavy_to_light_ratio', 'heavy_vehicle_pct']:
                features[col].fillna(features[col].mean(), inplace=True)
    
    # Add interaction features between merchant types and vehicle classes
    if 'merchant_name' in features.columns and 'vehicle_class_code' in features.columns:
        # Create key interaction features
        # 1. Get top merchants and top vehicle classes
        top_merchants = features['merchant_name'].value_counts().head(3).index.tolist()
        top_vcs = features['vehicle_class_code'].value_counts().head(3).index.tolist()
        
        # 2. Create interaction features for top combinations
        for m in top_merchants:
            merchant_col = f'merchant_{m}'
            for vc in top_vcs:
                vc_col = f'vc_{vc}'
                if merchant_col in features.columns and vc_col in features.columns:
                    # Create interaction feature
                    interaction_col = f'{merchant_col}_x_{vc_col}'
                    features[interaction_col] = features[merchant_col] * features[vc_col]
                    
                    # Also create percentage interaction
                    pct_interaction_col = f'{merchant_col}_pct_x_{vc_col}_pct'
                    if f'{merchant_col}_pct' in features.columns and f'{vc_col}_pct' in features.columns:
                        features[pct_interaction_col] = features[f'{merchant_col}_pct'] * features[f'{vc_col}_pct'] / 100
    
    # Drop NaN values created by lag and rolling features
    features.dropna(inplace=True)
    
    # Debug: output basic stats of target variable
    logging.debug("Target variable stats after enhanced feature engineering: mean=%.2f, std=%.2f", 
                  features['transaction_count'].mean(), features['transaction_count'].std())
    
    return features

# Assume ts_15min is your 15-minute aggregated DataFrame that already includes 'transaction_count'
ts_features = prepare_features_for_prediction(ts_15min)
logging.debug("Enhanced features shape: %s", ts_features.shape)

# -------------------------------
# 2.1.2 Improved Data Splitting and Feature Selection
# -------------------------------
def select_features_and_split(features_df):
    # Select features with different types for separate preprocessing
    time_cyclical_features = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos']
    categorical_features = ['is_weekend', 'quarter_of_day', 'day_part']
    numerical_features = ['hour', 'day_of_week']
    
    # Get lag and rolling features dynamically
    lag_features = [col for col in features_df.columns if col.startswith('lag_')]
    rolling_features = [col for col in features_df.columns if col.startswith('rolling_')]
    diff_features = [col for col in features_df.columns if col.startswith('tx_diff')]
    
    # Additional features if available
    additional_features = []
    if 'tx_per_second' in features_df.columns:
        additional_features.append('tx_per_second')
    if 'avg_amount_per_tx' in features_df.columns:
        additional_features.append('avg_amount_per_tx')
    
    # IMPROVED: Enhanced Merchant features - both counts and percentages
    merchant_count_features = [col for col in features_df.columns if col.startswith('merchant_') and not col.endswith('_pct')]
    merchant_pct_features = [col for col in features_df.columns if col.startswith('merchant_') and col.endswith('_pct')]
    if 'other_merchants' in features_df.columns:
        merchant_count_features.append('other_merchants')
    if 'other_merchants_pct' in features_df.columns:
        merchant_pct_features.append('other_merchants_pct')
    
    # IMPROVED: Enhanced Vehicle class features - both counts and percentages
    vehicle_count_features = [col for col in features_df.columns if col.startswith('vc_') and not col.endswith('_pct')]
    vehicle_pct_features = [col for col in features_df.columns if col.startswith('vc_') and col.endswith('_pct')]
    
    # Additional vehicle-related features
    vehicle_ratio_features = []
    if 'heavy_to_light_ratio' in features_df.columns:
        vehicle_ratio_features.append('heavy_to_light_ratio')
    if 'heavy_vehicle_pct' in features_df.columns:
        vehicle_ratio_features.append('heavy_vehicle_pct')
    
    # NEW: Interaction features between merchants and vehicle classes
    interaction_features = [col for col in features_df.columns if '_x_' in col]
    
    # Combine all features
    all_features = time_cyclical_features + categorical_features + numerical_features + \
                   lag_features + rolling_features + diff_features + additional_features + \
                   merchant_count_features + merchant_pct_features + \
                   vehicle_count_features + vehicle_pct_features + vehicle_ratio_features + \
                   interaction_features
    
    X = features_df[all_features]
    y = features_df['transaction_count']
    
    # Return features categorized for proper preprocessing
    feature_groups = {
        'time_cyclical': time_cyclical_features,
        'categorical': categorical_features,
        'numerical': numerical_features,
        'lag': lag_features,
        'rolling': rolling_features,
        'diff': diff_features,
        'additional': additional_features,
        'merchant_count': merchant_count_features,
        'merchant_pct': merchant_pct_features,
        'vehicle_count': vehicle_count_features,
        'vehicle_pct': vehicle_pct_features,
        'vehicle_ratio': vehicle_ratio_features,
        'interaction': interaction_features
    }
    
    return X, y, feature_groups

X, y, feature_groups = select_features_and_split(ts_features)
logging.debug("Selected features: %s", X.columns.tolist())

# -------------------------------
# 2.1.3 Improved Model Training with Hyperparameter Optimization & Evaluation
# -------------------------------
def create_preprocessor(feature_groups):
    """
    Create a column transformer to appropriately preprocess different feature types.
    Updated to handle merchant and vehicle features properly.
    """
    transformers = [
        ('time_cyclical', 'passthrough', feature_groups['time_cyclical']),
        ('categorical', 'passthrough', feature_groups['categorical']),
        ('numerical', StandardScaler(), feature_groups['numerical']),
        ('lag', RobustScaler(), feature_groups['lag']),
        ('rolling', RobustScaler(), feature_groups['rolling']),
        ('diff', RobustScaler(), feature_groups['diff'])
    ]
    
    # Add additional features if they exist
    if feature_groups['additional']:
        transformers.append(('additional', RobustScaler(), feature_groups['additional']))
    
    # IMPROVED: Add merchant count features if they exist
    if feature_groups['merchant_count']:
        transformers.append(('merchant_count', RobustScaler(), feature_groups['merchant_count']))
    
    # IMPROVED: Add merchant percentage features if they exist
    if feature_groups['merchant_pct']:
        transformers.append(('merchant_pct', 'passthrough', feature_groups['merchant_pct']))
    
    # IMPROVED: Add vehicle count features if they exist
    if feature_groups['vehicle_count']:
        transformers.append(('vehicle_count', RobustScaler(), feature_groups['vehicle_count']))
    
    # IMPROVED: Add vehicle percentage features if they exist
    if feature_groups['vehicle_pct']:
        transformers.append(('vehicle_pct', 'passthrough', feature_groups['vehicle_pct']))
    
    # Add vehicle ratio features if they exist
    if feature_groups['vehicle_ratio']:
        transformers.append(('vehicle_ratio', 'passthrough', feature_groups['vehicle_ratio']))
    
    # Add interaction features if they exist
    if feature_groups['interaction']:
        transformers.append(('interaction', RobustScaler(), feature_groups['interaction']))
    
    preprocessor = ColumnTransformer(transformers, remainder='drop')
    return preprocessor
def train_traffic_prediction_model(X, y, feature_groups):
    # Create preprocessor
    preprocessor = create_preprocessor(feature_groups)
    
    # Define time series split for proper evaluation
    tscv = TimeSeriesSplit(n_splits=3, test_size=12)
    
    # Define candidate model types with hyperparameter spaces for optimization
    model_params = {
        'XGBoost': {
            'model': XGBRegressor(random_state=42),
            'param_grid': {
                'regressor__n_estimators': [100, 200, 300],
                'regressor__learning_rate': [0.01, 0.05, 0.1],
                'regressor__max_depth': [3, 4, 5, 6],
                'regressor__subsample': [0.7, 0.8, 0.9],
                'regressor__colsample_bytree': [0.7, 0.8, 0.9],
                'regressor__gamma': [0, 1, 2],
                'regressor__reg_lambda': [1, 2, 5]
            }
        },
        'RandomForest': {
            'model': RandomForestRegressor(random_state=42),
            'param_grid': {
                'regressor__n_estimators': [100, 200, 300],
                'regressor__max_depth': [6, 8, 10, 12],
                'regressor__min_samples_split': [2, 5, 10],
                'regressor__min_samples_leaf': [2, 4, 6]
            }
        },
        'LightGBM': {
            'model': LGBMRegressor(random_state=42),
            'param_grid': {
                'regressor__n_estimators': [100, 200, 300],
                'regressor__learning_rate': [0.01, 0.05, 0.1],
                'regressor__max_depth': [3, 4, 5, 6],
                'regressor__subsample': [0.7, 0.8, 0.9],
                'regressor__colsample_bytree': [0.7, 0.8, 0.9],
                'regressor__reg_lambda': [1, 2, 5]
            }
        },
        'CatBoost': {
            'model': CatBoostRegressor(random_state=42, verbose=0),
            'param_grid': {
                'regressor__iterations': [100, 200, 300],
                'regressor__learning_rate': [0.01, 0.05, 0.1],
                'regressor__depth': [4, 6, 8],
                'regressor__l2_leaf_reg': [1, 3, 5, 7]
            }
        },
        'GradientBoosting': {
            'model': GradientBoostingRegressor(random_state=42),
            'param_grid': {
                'regressor__n_estimators': [100, 200, 300],
                'regressor__learning_rate': [0.01, 0.05, 0.1],
                'regressor__max_depth': [3, 4, 5],
                'regressor__subsample': [0.7, 0.8, 0.9],
                'regressor__min_samples_split': [2, 5, 10]
            }
        }
    }
    
    best_model = None
    best_score = float('-inf')
    best_metrics = None
    
    # Train and evaluate each model with hyperparameter optimization
    for name, model_config in model_params.items():
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', model_config['model'])
        ])
        
        logging.info(f"Optimizing hyperparameters for {name}...")
        
        random_search = RandomizedSearchCV(
            pipeline,
            param_distributions=model_config['param_grid'],
            n_iter=10,
            cv=tscv,
            scoring='r2',
            n_jobs=-1,
            random_state=42,
            verbose=1
        )
        
        random_search.fit(X, y)
        best_pipeline = random_search.best_estimator_
        best_params = random_search.best_params_
        
        logging.info(f"Best parameters for {name}: {best_params}")
        
        # Evaluate the best model using time series cross-validation
        mae_scores, rmse_scores, r2_scores = [], [], []
        for train_idx, test_idx in tscv.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
            best_pipeline.fit(X_train, y_train)
            y_pred = best_pipeline.predict(X_test)
            
            mae_scores.append(mean_absolute_error(y_test, y_pred))
            rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
            r2_scores.append(r2_score(y_test, y_pred))
        
        avg_mae = np.mean(mae_scores)
        avg_rmse = np.mean(rmse_scores)
        avg_r2 = np.mean(r2_scores)
        
        logging.info(f"{name} - Average MAE: {avg_mae:.2f}, RMSE: {avg_rmse:.2f}, R²: {avg_r2:.4f}")
        
        if avg_r2 > best_score:
            best_score = avg_r2
            best_model = best_pipeline
            best_metrics = {
                'name': name,
                'mae': avg_mae,
                'rmse': avg_rmse,
                'r2': avg_r2,
                'best_params': best_params
            }
    
    logging.info(f"Best model: {best_metrics['name']} with R²: {best_metrics['r2']:.4f}")
    best_model.fit(X, y)
    
    # Plot feature importance if supported
    if hasattr(best_model['regressor'], 'feature_importances_'):
        feature_names = X.columns
        importances = best_model['regressor'].feature_importances_
        sorted_idx = np.argsort(importances)[::-1]
        
        plt.figure(figsize=(12, 10))
        top_n = min(20, len(sorted_idx))
        plt.barh(range(top_n), importances[sorted_idx][:top_n])
        plt.yticks(range(top_n), [feature_names[i] for i in sorted_idx][:top_n])
        plt.title('Top 20 Feature Importance')
        plt.tight_layout()
        plt.savefig('images/feature_importance.png')
        plt.close()
    
    # Plot actual vs predicted for the last fold
    for train_idx, test_idx in tscv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    y_pred = best_model.predict(X_test)
    plt.figure(figsize=(16, 8))
    plt.subplot(2, 1, 1)
    plt.plot(y_test.values, label='Actual', marker='o', alpha=0.7)
    plt.plot(y_pred, label='Predicted', marker='x', alpha=0.7)
    plt.title(f'Traffic Volume: Actual vs Predicted ({best_metrics["name"]} Model)')
    plt.xlabel('Time Points')
    plt.ylabel('Transaction Count')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(2, 1, 2)
    errors = y_test.values - y_pred
    plt.bar(range(len(errors)), errors, color='red', alpha=0.6)
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.title('Prediction Errors')
    plt.xlabel('Time Points')
    plt.ylabel('Error (Actual - Predicted)')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('images/traffic_prediction_results.png')
    plt.close()
    
    return best_model, best_metrics

logging.info("Training improved traffic prediction model...")
traffic_model, metrics = train_traffic_prediction_model(X, y, feature_groups)
logging.info(f"Final Traffic Prediction metrics - MAE: {metrics['mae']:.2f}, RMSE: {metrics['rmse']:.2f}, R²: {metrics['r2']:.4f}")

# -------------------------------
# 2.2 Vehicle Class Distribution Prediction
# -------------------------------
logging.info("2.2 Improved Vehicle Class Distribution Prediction")

def create_vehicle_class_timeseries(df, interval='1H'):
    """
    Create time series data of vehicle class distribution.
    Returns absolute counts (vc_ts) and percentage distribution (vc_ts_pct).
    """
    vc_ts = df.set_index('initiated_time').groupby([pd.Grouper(freq=interval), 'vehicle_class_code']).size().unstack(fill_value=0)
    vc_ts_pct = vc_ts.div(vc_ts.sum(axis=1), axis=0) * 100
    return vc_ts, vc_ts_pct

# Create vehicle class time series
vc_hourly, vc_hourly_pct = create_vehicle_class_timeseries(df, '1H')
logging.debug("Vehicle class distribution (percentage) shape: %s", vc_hourly_pct.shape)

# Plot vehicle class distribution
plt.figure(figsize=(14, 8))
vc_hourly_pct.plot(kind='area', stacked=True, colormap='viridis')
plt.title('Vehicle Class Distribution Over Time')
plt.xlabel('Time')
plt.ylabel('Percentage of Vehicles')
plt.legend(title='Vehicle Class', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('images/vehicle_class_distribution.png')
plt.close()

# -------------------------------
# 2.2.1 Vehicle Class Prediction with Hyperparameter Optimization
# -------------------------------
def train_vehicle_class_prediction_model(vc_hourly, df):
    vc_features = vc_hourly.reset_index()
    
    # Enhanced time features
    vc_features['hour'] = vc_features['initiated_time'].dt.hour
    vc_features['day_of_week'] = vc_features['initiated_time'].dt.dayofweek
    vc_features['is_weekend'] = vc_features['day_of_week'].isin([5, 6]).astype(int)
    vc_features['day_part'] = pd.cut(vc_features['hour'], 
                                     bins=[0, 6, 12, 18, 24], 
                                     labels=['night', 'morning', 'afternoon', 'evening'])
    vc_features['day_part'] = vc_features['day_part'].astype('category').cat.codes
    
    # Cyclical features
    vc_features['hour_sin'] = np.sin(2 * np.pi * vc_features['hour'] / 24)
    vc_features['hour_cos'] = np.cos(2 * np.pi * vc_features['hour'] / 24)
    vc_features['day_sin'] = np.sin(2 * np.pi * vc_features['day_of_week'] / 7)
    vc_features['day_cos'] = np.cos(2 * np.pi * vc_features['day_of_week'] / 7)
    
    # Weather features (if available)
    weather_features = []
    if 'temperature' in df.columns:
        temp_hourly = df.set_index('initiated_time')['temperature'].resample('1H').mean()
        vc_features = vc_features.join(temp_hourly, on='initiated_time')
        weather_features.append('temperature')
    if 'precipitation' in df.columns:
        precip_hourly = df.set_index('initiated_time')['precipitation'].resample('1H').sum()
        vc_features = vc_features.join(precip_hourly, on='initiated_time')
        weather_features.append('precipitation')
    
    # NEW: Enhanced merchant name features - better handling
    merchant_features = []
    if 'merchant_name' in df.columns:
        # Get the most common merchant names (top 5)
        top_merchants = df['merchant_name'].value_counts().head(5).index.tolist()
        
        # Create one-hot encoded columns for each merchant
        merchant_dummies = pd.get_dummies(df['merchant_name'], prefix='merchant')
        merchant_dummies['initiated_time'] = df['initiated_time']
        
        # Aggregate by hour using sum (to get counts) then divide by total to get percentages
        merchant_hourly = merchant_dummies.set_index('initiated_time').groupby(pd.Grouper(freq='1H')).sum()
        totals = merchant_hourly.sum(axis=1)
        merchant_hourly_pct = merchant_hourly.div(totals, axis=0) * 100
        
        # Only keep columns for top merchants
        top_merchant_cols = [f'merchant_{m}' for m in top_merchants if f'merchant_{m}' in merchant_hourly_pct.columns]
        if top_merchant_cols:
            merchant_hourly_pct = merchant_hourly_pct[top_merchant_cols]
            
            # Join these features with the main features dataframe
            vc_features = vc_features.join(merchant_hourly_pct, on='initiated_time')
            
            # Fill NAs with zeros (no transactions for that merchant in that hour)
            for col in top_merchant_cols:
                if col in vc_features.columns:
                    vc_features[col].fillna(0, inplace=True)
            
            merchant_features = top_merchant_cols
    
    # Direction features (if available)
    direction_features = []
    if 'direction' in df.columns:
        # Get unique direction values (N, S, E, W)
        direction_dummies = pd.get_dummies(df['direction'], prefix='direction')
        direction_dummies['initiated_time'] = df['initiated_time']
        
        # Aggregate by hour using sum (to get counts) then divide by total to get percentages
        direction_hourly = direction_dummies.set_index('initiated_time').groupby(pd.Grouper(freq='1H')).sum()
        totals = direction_hourly.sum(axis=1)
        direction_hourly_pct = direction_hourly.div(totals, axis=0) * 100
        
        vc_features = vc_features.join(direction_hourly_pct, on='initiated_time')
        
        direction_features = direction_hourly_pct.columns.tolist()
    
    # Feature selection
    X_cols = ['hour', 'day_of_week', 'is_weekend', 'day_part',
              'hour_sin', 'hour_cos', 'day_sin', 'day_cos'] + weather_features + merchant_features + direction_features
    X = vc_features[X_cols]
    y = vc_hourly.values  # each column corresponds to a vehicle class
    
    X = X.dropna(axis=1)
    
    # Create column transformer for different feature types
    numeric_features = ['hour', 'day_of_week'] + weather_features
    cyclical_features = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos']
    categorical_features = ['is_weekend', 'day_part']
    distribution_features = merchant_features + direction_features
    
    preprocessor = ColumnTransformer([
        ('cyclical', 'passthrough', cyclical_features),
        ('numerical', StandardScaler(), numeric_features),
        ('categorical', 'passthrough', categorical_features),
        ('distribution', 'passthrough', distribution_features)
    ])
    
    # Create time series split for proper validation
    tscv = TimeSeriesSplit(n_splits=3, test_size=6)
    
    # Define models with hyperparameter spaces for optimization.
    # Note: Since y is multi-output, we wrap each estimator with MultiOutputRegressor.
    # The parameter grid keys are updated to pass parameters to the underlying estimator.
    models = {
        'RandomForest': {
            'model': RandomForestRegressor(random_state=42),
            'param_grid': {
                'regressor__estimator__n_estimators': [100, 200, 300],
                'regressor__estimator__max_depth': [6, 8, 10],
                'regressor__estimator__min_samples_split': [2, 5, 10]
            }
        },
        'XGBoost': {
            'model': XGBRegressor(random_state=42),
            'param_grid': {
                'regressor__estimator__n_estimators': [100, 200],
                'regressor__estimator__learning_rate': [0.01, 0.05, 0.1],
                'regressor__estimator__max_depth': [3, 4, 5]
            }
        },
        'GradientBoosting': {
            'model': GradientBoostingRegressor(random_state=42),
            'param_grid': {
                'regressor__estimator__n_estimators': [100, 200],
                'regressor__estimator__learning_rate': [0.01, 0.05, 0.1],
                'regressor__estimator__max_depth': [3, 4, 5]
            }
        }
    }
    
    best_model = None
    best_score = float('inf')  # minimize MAE
    best_metrics = None
    
    for name, model_config in models.items():
        # Wrap the model with MultiOutputRegressor
        base_model = model_config['model']
        multioutput_model = MultiOutputRegressor(base_model)
        
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', multioutput_model)
        ])
        
        logging.info(f"Optimizing hyperparameters for {name} in vehicle class prediction...")
        random_search = RandomizedSearchCV(
            pipeline,
            param_distributions=model_config['param_grid'],
            n_iter=10,
            cv=tscv,
            scoring='neg_mean_absolute_error',
            n_jobs=-1,
            random_state=42,
            verbose=1
        )
        random_search.fit(X, y)
        best_pipeline = random_search.best_estimator_
        best_params = random_search.best_params_
        logging.info(f"Best parameters for {name}: {best_params}")
        
        mae_scores, rmse_scores = [], []
        for train_idx, test_idx in tscv.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            best_pipeline.fit(X_train, y_train)
            y_pred = best_pipeline.predict(X_test)
            mae_scores.append(mean_absolute_error(y_test, y_pred))
            rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
        avg_mae = np.mean(mae_scores)
        avg_rmse = np.mean(rmse_scores)
        logging.info(f"{name} - Average MAE: {avg_mae:.2f}, RMSE: {avg_rmse:.2f}")
        
        if avg_mae < best_score:
            best_score = avg_mae
            best_model = best_pipeline
            best_metrics = {
                'name': name,
                'mae': avg_mae,
                'rmse': avg_rmse,
                'best_params': best_params
            }
    
    logging.info(f"Best vehicle class model: {best_metrics['name']} with MAE: {best_metrics['mae']:.2f}")
    best_model.fit(X, y)
    
    # Build feature vector for next hour prediction
    last_date = vc_features['initiated_time'].max()
    next_hour = last_date + pd.Timedelta(hours=1)
    next_x_dict = {
        'hour': [next_hour.hour],
        'day_of_week': [next_hour.dayofweek],
        'is_weekend': [1 if next_hour.dayofweek >= 5 else 0],
        'day_part': [next_hour.hour // 6],
        'hour_sin': [np.sin(2 * np.pi * next_hour.hour / 24)],
        'hour_cos': [np.cos(2 * np.pi * next_hour.hour / 24)],
        'day_sin': [np.sin(2 * np.pi * next_hour.dayofweek / 7)],
        'day_cos': [np.cos(2 * np.pi * next_hour.dayofweek / 7)]
    }
    for feature in weather_features:
        if feature in X.columns:
            next_x_dict[feature] = [X[feature].iloc[-1]]
    for feature in merchant_features + direction_features:
        if feature in X.columns:
            next_x_dict[feature] = [X[feature].iloc[-1]]
    
    next_x = pd.DataFrame(next_x_dict)
    next_x = next_x[[col for col in next_x.columns if col in X.columns]]
    
    next_distribution = best_model.predict(next_x)[0]
    predicted_distribution = pd.Series(next_distribution, index=vc_hourly.columns)
    
    logging.info(f"Predicted vehicle class distribution for {next_hour}:")
    logging.info(predicted_distribution)
    
    with open('vehicle_class_prediction.txt', 'w') as f:
        f.write(f"Predicted vehicle class distribution for {next_hour}:\n")
        f.write(predicted_distribution.to_string())
    
    # Visualize predictions vs. actuals for the last fold
    last_train_idx, last_test_idx = list(tscv.split(X))[-1]
    X_train, X_test = X.iloc[last_train_idx], X.iloc[last_test_idx]
    y_train, y_test = y[last_train_idx], y[last_test_idx]
    
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)
    y_test_df = pd.DataFrame(y_test, columns=vc_hourly.columns)
    y_pred_df = pd.DataFrame(y_pred, columns=vc_hourly.columns)
    
    top_classes = vc_hourly.sum().sort_values(ascending=False).head(5).index.tolist()
    plt.figure(figsize=(16, 10))
    for i, vc in enumerate(top_classes):
        plt.subplot(len(top_classes), 1, i+1)
        plt.plot(y_test_df.index, y_test_df[vc], 'b-', label=f'Actual {vc}')
        plt.plot(y_test_df.index, y_pred_df[vc], 'r--', label=f'Predicted {vc}')
        plt.ylabel('Count')
        plt.legend()
        if i == 0:
            plt.title('Top Vehicle Classes: Actual vs. Predicted')
        if i == len(top_classes)-1:
            plt.xlabel('Time Points')
    
    plt.tight_layout()
    plt.savefig('images/vehicle_class_prediction_results.png')
    plt.close()
    
    return best_model, best_metrics, predicted_distribution

logging.info("Training improved vehicle class prediction model...")
if not vc_hourly.empty:
    vc_model, vc_metrics, predicted_distribution = train_vehicle_class_prediction_model(vc_hourly, df)
else:
    logging.warning("No vehicle class data available for modeling")


INFO: 2. IMPROVED PREDICTIVE MODELS DEVELOPMENT
INFO: 2.1 Traffic Volume Prediction Model
DEBUG: Target variable stats after enhanced feature engineering: mean=3910.81, std=1215.02
DEBUG: Enhanced features shape: (91, 27)
DEBUG: Selected features: ['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'is_weekend', 'quarter_of_day', 'day_part', 'hour', 'day_of_week', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'rolling_mean_3', 'rolling_std_3', 'rolling_mean_6', 'rolling_max_6', 'rolling_min_6', 'tx_diff_1', 'tx_diff_2', 'tx_per_second', 'avg_amount_per_tx']
INFO: Training improved traffic prediction model...
INFO: Optimizing hyperparameters for XGBoost...


Fitting 3 folds for each of 10 candidates, totalling 30 fits


INFO: Best parameters for XGBoost: {'regressor__subsample': 0.9, 'regressor__reg_lambda': 2, 'regressor__n_estimators': 300, 'regressor__max_depth': 4, 'regressor__learning_rate': 0.1, 'regressor__gamma': 0, 'regressor__colsample_bytree': 0.8}
INFO: XGBoost - Average MAE: 227.87, RMSE: 291.58, R²: 0.1005
INFO: Optimizing hyperparameters for RandomForest...


Fitting 3 folds for each of 10 candidates, totalling 30 fits


INFO: Best parameters for RandomForest: {'regressor__n_estimators': 300, 'regressor__min_samples_split': 2, 'regressor__min_samples_leaf': 2, 'regressor__max_depth': 12}
INFO: RandomForest - Average MAE: 197.85, RMSE: 262.52, R²: 0.2059
INFO: Optimizing hyperparameters for LightGBM...


...

Fitting 3 folds for each of 10 candidates, totalling 30 fits


INFO: Best parameters for XGBoost: {'regressor__estimator__n_estimators': 200, 'regressor__estimator__max_depth': 3, 'regressor__estimator__learning_rate': 0.1}
INFO: XGBoost - Average MAE: 418.95, RMSE: 1373.56
INFO: Optimizing hyperparameters for GradientBoosting in vehicle class prediction...


Fitting 3 folds for each of 10 candidates, totalling 30 fits


INFO: Best parameters for GradientBoosting: {'regressor__estimator__n_estimators': 200, 'regressor__estimator__max_depth': 4, 'regressor__estimator__learning_rate': 0.01}
INFO: GradientBoosting - Average MAE: 345.30, RMSE: 1157.56
INFO: Best vehicle class model: RandomForest with MAE: 340.04
INFO: Predicted vehicle class distribution for 2024-03-20 00:00:00:
INFO: vehicle_class_code
VC10     841.296583
VC11     467.175500
VC12     824.637262
VC13     533.828333
VC14     125.443833
VC15       4.578000
VC16       0.350000
VC20    1045.712000
VC4     7708.670000
VC5      763.218917
VC6        0.000000
VC7      670.065000
VC8       36.293333
VC9       54.432714
dtype: float64


<Figure size 1400x800 with 0 Axes>

In [ ]:
# 3. ANOMALY DETECTION SYSTEM
print("\n3. ANOMALY DETECTION SYSTEM")
print("-" * 50)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.cluster import KMeans, DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

# 3.1 Traffic Pattern Anomaly Detection
print("\n3.1 Traffic Pattern Anomaly Detection")

def build_traffic_anomaly_detector(ts_data):
    """Build an anomaly detection model for traffic patterns using multiple methods."""
    # Prepare features
    features = ts_data.copy()
    features['hour'] = features['initiated_time'].dt.hour
    features['day_of_week'] = features['initiated_time'].dt.dayofweek
    features['is_weekend'] = features['day_of_week'].isin([5, 6]).astype(int)
    features['month'] = features['initiated_time'].dt.month
    features['day'] = features['initiated_time'].dt.day
    
    # Features for anomaly detection
    X = features[['transaction_count', 'hour', 'day_of_week', 'is_weekend', 'month', 'day']]
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Initialize dictionary to store models and results
    models = {}
    
    # 1. Isolation Forest model
    print("\nTraining Isolation Forest model...")
    if_model = IsolationForest(
        contamination=0.05,  # Expected proportion of anomalies
        n_estimators=200,    # Increased number of trees
        max_samples='auto',
        random_state=42
    )
    
    if_model.fit(X_scaled)
    
    # Predict anomalies
    features['if_anomaly'] = if_model.predict(X_scaled)
    features['if_anomaly_score'] = if_model.score_samples(X_scaled)
    features['if_is_anomaly'] = features['if_anomaly'].apply(lambda x: 1 if x == -1 else 0)
    
    # Store model in dictionary
    models['isolation_forest'] = (if_model, features[features['if_is_anomaly'] == 1])
    
    # 2. DBSCAN clustering for anomaly detection
    print("Training DBSCAN model...")
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    features['dbscan_cluster'] = dbscan.fit_predict(X_scaled)
    
    # Mark outliers (cluster -1) as anomalies
    features['dbscan_is_anomaly'] = (features['dbscan_cluster'] == -1).astype(int)
    
    # Store model in dictionary
    models['dbscan'] = (dbscan, features[features['dbscan_is_anomaly'] == 1])
    
    # Create ensemble anomaly detection (combine results from both methods)
    features['ensemble_score'] = features['if_is_anomaly'] + features['dbscan_is_anomaly']
    features['ensemble_is_anomaly'] = (features['ensemble_score'] > 0).astype(int)
    
    # Count anomalies for each method
    if_anomaly_count = features['if_is_anomaly'].sum()
    dbscan_anomaly_count = features['dbscan_is_anomaly'].sum()
    ensemble_anomaly_count = features['ensemble_is_anomaly'].sum()
    
    print(f"Isolation Forest: Detected {if_anomaly_count} anomalies out of {len(features)} intervals")
    print(f"DBSCAN: Detected {dbscan_anomaly_count} anomalies out of {len(features)} intervals")
    print(f"Ensemble: Detected {ensemble_anomaly_count} anomalies out of {len(features)} intervals")
    
    # Visualize anomalies for each method
    plt.figure(figsize=(18, 12))
    
    # Isolation Forest
    plt.subplot(2, 2, 1)
    plt.scatter(features['initiated_time'], features['transaction_count'], 
                c=features['if_is_anomaly'], cmap='coolwarm', alpha=0.6)
    plt.title('Isolation Forest Anomaly Detection')
    plt.xlabel('Time')
    plt.ylabel('Transaction Count')
    plt.colorbar(label='Anomaly (1) / Normal (0)')
    
    # DBSCAN
    plt.subplot(2, 2, 2)
    plt.scatter(features['initiated_time'], features['transaction_count'], 
                c=features['dbscan_is_anomaly'], cmap='coolwarm', alpha=0.6)
    plt.title('DBSCAN Anomaly Detection')
    plt.xlabel('Time')
    plt.ylabel('Transaction Count')
    plt.colorbar(label='Anomaly (1) / Normal (0)')
    
    # Ensemble
    plt.subplot(2, 2, 3)
    plt.scatter(features['initiated_time'], features['transaction_count'], 
                c=features['ensemble_is_anomaly'], cmap='coolwarm', alpha=0.6)
    plt.title('Ensemble Anomaly Detection')
    plt.xlabel('Time')
    plt.ylabel('Transaction Count')
    plt.colorbar(label='Anomaly (1) / Normal (0)')
    
    plt.tight_layout()
    plt.savefig('images/traffic_anomalies_comparison.png')
    plt.close()
    
    # Return models and anomalies
    ensemble_anomalies = features[features['ensemble_is_anomaly'] == 1]
    return models, scaler, ensemble_anomalies, features

# Build traffic anomaly detector
print("\nBuilding traffic anomaly detection models...")
anomaly_models, anomaly_scaler, anomalies, features_with_anomalies = build_traffic_anomaly_detector(ts_15min)

# Show the detected anomalies
print("\nTop 10 detected traffic anomalies:")
print(anomalies[['initiated_time', 'transaction_count']].sort_values('transaction_count', ascending=False).head(10))

# 3.2 Toll Skipping Detection
print("\n3.2 Toll Skipping Detection")

def detect_potential_toll_skipping(df):
    """Detect potential toll skipping patterns."""
    # Print columns for debugging
    print("Available columns in dataframe:")
    print(df.columns.tolist())
    
    # Check if the vehicle registration column exists
    vehicle_col = 'vehicle_regn_number'
    
    if vehicle_col not in df.columns:
        # Try to find alternatives
        possible_columns = [col for col in df.columns if any(term in col.lower() for term in 
                           ['vehicle', 'regn', 'registration', 'reg', 'tag', 'transponder', 'id'])]
        
        if possible_columns:
            vehicle_col = possible_columns[0]
            print(f"Using alternative column: '{vehicle_col}'")
        else:
            print("No suitable vehicle identifier found. Creating synthetic ID.")
            df['synthetic_vehicle_id'] = range(1, len(df) + 1)
            vehicle_col = 'synthetic_vehicle_id'
    
    print(f"Using '{vehicle_col}' as vehicle identifier")
    
    # Group by vehicle identifier
    print(f"Grouping by {vehicle_col}...")
    vehicle_trips = df.groupby(vehicle_col).agg({
        'initiated_time': ['min', 'max', 'count'],
        'merchant_name': 'nunique',
        'txn_amount': 'sum'
    })
    
    vehicle_trips.columns = ['first_seen', 'last_seen', 'total_trips', 'unique_plazas', 'total_amount']
    
    # Calculate time difference between first and last trip
    vehicle_trips['time_span_hours'] = (vehicle_trips['last_seen'] - vehicle_trips['first_seen']).dt.total_seconds() / 3600
    
    # Calculate average transactions per hour active
    vehicle_trips['trips_per_hour'] = vehicle_trips['total_trips'] / vehicle_trips['time_span_hours'].clip(lower=1)
    
    # Calculate average amount per trip
    vehicle_trips['avg_amount_per_trip'] = vehicle_trips['total_amount'] / vehicle_trips['total_trips']
    
    # Calculate ratio of unique plazas to total trips
    vehicle_trips['plaza_to_trip_ratio'] = vehicle_trips['unique_plazas'] / vehicle_trips['total_trips']
    
    # Filter for vehicles with multiple trips
    multi_trip_vehicles = vehicle_trips[vehicle_trips['total_trips'] > 1].copy()
    
    models = {}
    
    if len(multi_trip_vehicles) > 0:
        print(f"Found {len(multi_trip_vehicles)} vehicles with multiple trips")
        
        # Features for clustering
        features = multi_trip_vehicles[['total_trips', 'unique_plazas', 'trips_per_hour', 
                                       'avg_amount_per_trip', 'plaza_to_trip_ratio']]
        
        # Scale features
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)
        
        # Try multiple clustering algorithms
        
        # 1. K-Means clustering
        print("\nPerforming K-Means clustering...")
        # Determine optimal number of clusters
        inertia = []
        k_range = range(1, min(11, len(features_scaled)))
        for k in k_range:
            kmeans = KMeans(n_clusters=k, random_state=42)
            kmeans.fit(features_scaled)
            inertia.append(kmeans.inertia_)
        
        # Plot elbow curve
        plt.figure(figsize=(10, 6))
        plt.plot(k_range, inertia, 'o-')
        plt.xlabel('Number of Clusters')
        plt.ylabel('Inertia')
        plt.title('Elbow Method for Optimal k')
        plt.grid(True)
        plt.savefig('images/kmeans_elbow_curve.png')
        plt.close()
        
        # Choose optimal k (simplified method)
        optimal_k = 3
        if len(inertia) > 2:
            for i in range(1, len(inertia)-1):
                if (inertia[i-1] - inertia[i]) > 3 * (inertia[i] - inertia[i+1]):
                    optimal_k = i+1
                    break
        
        print(f"Optimal number of clusters determined: {optimal_k}")
        
        # Perform KMeans clustering
        kmeans = KMeans(n_clusters=optimal_k, random_state=42)
        multi_trip_vehicles['kmeans_cluster'] = kmeans.fit_predict(features_scaled)
        
        # Store model
        models['kmeans'] = kmeans
        joblib.dump(kmeans, 'models/toll_kmeans.pkl')
        # 2. DBSCAN clustering
        print("Performing DBSCAN clustering...")
        dbscan = DBSCAN(eps=0.5, min_samples=5)
        multi_trip_vehicles['dbscan_cluster'] = dbscan.fit_predict(features_scaled)
        
        # Store model
        models['dbscan'] = dbscan
        
        # Analyze KMeans clusters - FIXED: use count() instead of vehicle_col
        kmeans_cluster_stats = multi_trip_vehicles.groupby('kmeans_cluster').agg({
            'total_trips': 'mean',
            'unique_plazas': 'mean',
            'trips_per_hour': 'mean',
            'avg_amount_per_trip': 'mean',
            'plaza_to_trip_ratio': 'mean',
            'total_amount': 'count'  # Just count the rows instead of using the vehicle column
        })
        
        # Rename the count column to 'vehicle_count'
        kmeans_cluster_stats = kmeans_cluster_stats.rename(columns={'total_amount': 'vehicle_count'})
        
        print("\nK-Means Cluster statistics:")
        print(kmeans_cluster_stats)
        
        # Identify potential toll skipping clusters
        toll_skip_score = kmeans_cluster_stats['plaza_to_trip_ratio'] 
        potential_skip_cluster = toll_skip_score.idxmin()
        
        print(f"\nPotential toll skipping K-Means cluster identified: Cluster {potential_skip_cluster}")
        print(f"This cluster has {kmeans_cluster_stats.loc[potential_skip_cluster, 'vehicle_count']} vehicles")
        
        # Get vehicles in the potential toll skipping cluster from KMeans
        kmeans_skip_vehicles = multi_trip_vehicles[multi_trip_vehicles['kmeans_cluster'] == potential_skip_cluster]
        
        # Analyze DBSCAN outliers (cluster -1)
        if -1 in multi_trip_vehicles['dbscan_cluster'].unique():
            dbscan_outliers = multi_trip_vehicles[multi_trip_vehicles['dbscan_cluster'] == -1]
            print(f"\nDBSCAN identified {len(dbscan_outliers)} potential outlier vehicles")
            
            # Combine results from both methods
            combined_suspects = pd.Index(list(set(kmeans_skip_vehicles.index) | set(dbscan_outliers.index)))
            print(f"\nCombined unique suspect vehicles from both methods: {len(combined_suspects)}")
            
            # Create a combined anomaly score
            multi_trip_vehicles['combined_score'] = 0
            multi_trip_vehicles.loc[multi_trip_vehicles['kmeans_cluster'] == potential_skip_cluster, 'combined_score'] += 1
            multi_trip_vehicles.loc[multi_trip_vehicles['dbscan_cluster'] == -1, 'combined_score'] += 1
            
            # Get the most suspicious vehicles (detected by both methods)
            high_confidence_suspects = multi_trip_vehicles[multi_trip_vehicles['combined_score'] > 1]
            print(f"High confidence suspects (detected by both methods): {len(high_confidence_suspects)}")
            
            return high_confidence_suspects, models, scaler
        else:
            print("DBSCAN did not identify any outliers")
            return kmeans_skip_vehicles, models, scaler
    else:
        print("Not enough multi-trip vehicles to perform clustering")
        return None, None, None

# Detect potential toll skipping
print("\nDetecting potential toll skipping patterns...")
potential_skip_vehicles, skip_models, skip_scaler = detect_potential_toll_skipping(df)

if potential_skip_vehicles is not None and len(potential_skip_vehicles) > 0:
    print("\nSample of potential toll skipping vehicles:")
    print(potential_skip_vehicles.head())
    
    # Create a vehicle behavior classifier
    print("\n3.3 Building supervised vehicle behavior classifier")
    
    # Use our detected vehicles to create a synthetic labeled dataset
    print("Creating synthetic labeled dataset for demonstration...")
    
    # Extract features from all multi-trip vehicles - use the index directly
    all_vehicles = potential_skip_vehicles.copy()
    
    # Create more "normal" vehicles by slightly modifying the suspicious ones
    normal_vehicles = potential_skip_vehicles.copy()
    # Make them less suspicious by increasing plaza_to_trip_ratio
    normal_vehicles['plaza_to_trip_ratio'] = normal_vehicles['plaza_to_trip_ratio'] * 1.5
    normal_vehicles.index = [f"normal_{idx}" for idx in normal_vehicles.index]
    
    all_vehicles = pd.concat([all_vehicles, normal_vehicles])
    
    # Create labels (1 for suspicious, 0 for normal)
    all_vehicles['label'] = 0
    all_vehicles.loc[potential_skip_vehicles.index, 'label'] = 1
    
    # Extract features
    X = all_vehicles[['total_trips', 'unique_plazas', 'trips_per_hour', 
                     'avg_amount_per_trip', 'plaza_to_trip_ratio']]
    y = all_vehicles['label']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Train a RandomForest classifier
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_classifier.fit(X_train, y_train)
    
    # Evaluate the classifier
    y_pred = rf_classifier.predict(X_test)
    
    print("\nVehicle behavior classifier metrics:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': rf_classifier.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nFeature importance for detecting suspicious vehicles:")
    print(feature_importance)
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['Feature'], feature_importance['Importance'])
    plt.xlabel('Importance')
    plt.title('Feature Importance for Suspicious Vehicle Detection')
    plt.tight_layout()
    plt.savefig('images/feature_importance.png')
    plt.close()
    
    # Save the classifier
    vehicle_classifier = rf_classifier
else:
    print("No potential toll skipping vehicles detected. Skipping classifier creation.")

# 3.4 Enhanced Traffic Pattern Analysis
print("\n3.4 Enhanced Traffic Pattern Analysis")

def analyze_traffic_patterns_with_anomalies(features):
    """Analyze traffic patterns incorporating anomaly information."""
    # Group by hour of day
    hourly_patterns = features.groupby('hour').agg({
        'transaction_count': 'mean',
        'ensemble_is_anomaly': 'mean'  # Gives the proportion of anomalies by hour
    }).reset_index()
    
    # Group by day of week
    daily_patterns = features.groupby('day_of_week').agg({
        'transaction_count': 'mean',
        'ensemble_is_anomaly': 'mean'  # Gives the proportion of anomalies by day
    }).reset_index()
    
    # Visualize hourly patterns
    plt.figure(figsize=(18, 10))
    
    plt.subplot(2, 2, 1)
    plt.bar(hourly_patterns['hour'], hourly_patterns['transaction_count'])
    plt.title('Average Transactions by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Average Transaction Count')
    plt.xticks(range(0, 24, 2))
    plt.grid(True, alpha=0.3)
    
    plt.subplot(2, 2, 2)
    plt.bar(hourly_patterns['hour'], hourly_patterns['ensemble_is_anomaly'] * 100)
    plt.title('Percentage of Anomalies by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Anomaly Percentage')
    plt.xticks(range(0, 24, 2))
    plt.grid(True, alpha=0.3)
    
    plt.subplot(2, 2, 3)
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    plt.bar(days, daily_patterns['transaction_count'])
    plt.title('Average Transactions by Day of Week')
    plt.xlabel('Day of Week')
    plt.ylabel('Average Transaction Count')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    plt.subplot(2, 2, 4)
    plt.bar(days, daily_patterns['ensemble_is_anomaly'] * 100)
    plt.title('Percentage of Anomalies by Day of Week')
    plt.xlabel('Day of Week')
    plt.ylabel('Anomaly Percentage')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('images/traffic_pattern_analysis.png')
    plt.close()
    
    # Find the most anomalous hours and days
    most_anomalous_hours = hourly_patterns.sort_values('ensemble_is_anomaly', ascending=False)[['hour', 'ensemble_is_anomaly']].head(3)
    most_anomalous_days = daily_patterns.sort_values('ensemble_is_anomaly', ascending=False)[['day_of_week', 'ensemble_is_anomaly']].head(3)
    
    print("\nMost anomalous hours of the day:")
    for _, row in most_anomalous_hours.iterrows():
        print(f"Hour {int(row['hour'])}: {row['ensemble_is_anomaly']*100:.2f}% anomalies")
    
    print("\nMost anomalous days of the week:")
    day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
    for _, row in most_anomalous_days.iterrows():
        day = day_names[row['day_of_week']]
        print(f"{day}: {row['ensemble_is_anomaly']*100:.2f}% anomalies")
    
    return hourly_patterns, daily_patterns

# Analyze traffic patterns including anomaly information
if 'features_with_anomalies' in locals():
    print("\nAnalyzing traffic patterns with anomaly information...")
    hourly_patterns, daily_patterns = analyze_traffic_patterns_with_anomalies(features_with_anomalies)



In [ ]:
# # 3. ENHANCED ANOMALY DETECTION SYSTEM
# print("\n3. ENHANCED ANOMALY DETECTION SYSTEM")
# print("-" * 50)

# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.preprocessing import StandardScaler, RobustScaler
# from sklearn.ensemble import IsolationForest, RandomForestClassifier
# from sklearn.cluster import KMeans, DBSCAN, OPTICS
# from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
# from sklearn.svm import OneClassSVM
# from sklearn.neighbors import LocalOutlierFactor
# from sklearn.pipeline import Pipeline
# from sklearn.feature_selection import SelectFromModel
# from sklearn.decomposition import PCA
# import xgboost as xgb
# from lightgbm import LGBMClassifier
# import joblib
# from datetime import datetime, timedelta

# # Set style for better visualizations
# plt.style.use('seaborn-v0_8-whitegrid')
# sns.set_palette('viridis')

# # 3.1 Advanced Traffic Pattern Anomaly Detection
# print("\n3.1 Advanced Traffic Pattern Anomaly Detection")
# # Add this function to calculate rolling features in real-time
# def calculate_rolling_features(current_value, historical_values):
#     if len(historical_values) >= 12:
#         window = historical_values[-12:]
#     else:
#         window = historical_values
    
#     return {
#         'rolling_mean_3h': np.mean(window) if window else current_value,
#         'rolling_std_3h': np.std(window) if window else 0,
#         'z_score': (current_value - np.mean(window)) / (np.std(window) + 1e-6)
#     }
# def build_traffic_anomaly_detector(ts_data, contamination=0.05):

#     # Create more sophisticated features
#     features = ts_data.copy()
    
#     # Temporal features
#     features['hour'] = features['initiated_time'].dt.hour
#     features['day_of_week'] = features['initiated_time'].dt.dayofweek
#     features['is_weekend'] = features['day_of_week'].isin([5, 6]).astype(int)
#     features['month'] = features['initiated_time'].dt.month
#     features['day'] = features['initiated_time'].dt.day
#     features['quarter_of_day'] = features['hour'] // 6
#     features['is_business_hours'] = ((features['hour'] >= 9) & (features['hour'] <= 17) & 
#                                      (~features['is_weekend'])).astype(int)
    
#     # Rolling statistics (if time series is sorted)
#     if 'transaction_count' in features.columns:
#         features['rolling_mean_3h'] = features['transaction_count'].rolling(window=12, min_periods=1).mean()
#         features['rolling_std_3h'] = features['transaction_count'].rolling(window=12, min_periods=1).std()
#         features['z_score'] = (features['transaction_count'] - features['rolling_mean_3h']) / features['rolling_std_3h'].replace(0, 1)
    
#     # Features for anomaly detection
#     # Update the feature list in build_traffic_anomaly_detector
#     base_features = [
#         'transaction_count', 
#         'hour', 
#         'day_of_week', 
#         'is_weekend',
#         'month', 
#         'day',
#         'quarter_of_day',
#         'is_business_hours',
#         'rolling_mean_3h',
#         'rolling_std_3h',
#         'z_score'
#     ]
    
#     # Add rolling features if they exist
#     anomaly_features = base_features.copy()
#     for col in ['rolling_mean_3h', 'rolling_std_3h', 'z_score']:
#         if col in features.columns:
#             anomaly_features.append(col)
    
#     X = features[anomaly_features].fillna(0)
    
#     # Use RobustScaler to handle outliers better
#     scaler = RobustScaler()
#     X_scaled = scaler.fit_transform(X)
    
#     # Initialize dictionary to store models and results
#     models = {}
    
#     # 1. Isolation Forest model with optimized parameters
#     print("\nTraining Isolation Forest model...")
#     if_model = IsolationForest(
#         contamination=contamination,
#         n_estimators=300,     # More trees for better accuracy
#         max_samples='auto',
#         bootstrap=True,       # Enable bootstrapping for more robust results
#         n_jobs=-1,            # Use all available cores
#         random_state=42
#     )
    
#     if_model.fit(X_scaled)
    
#     # Predict anomalies
#     features['if_anomaly'] = if_model.predict(X_scaled)
#     features['if_anomaly_score'] = if_model.score_samples(X_scaled)
#     features['if_is_anomaly'] = features['if_anomaly'].apply(lambda x: 1 if x == -1 else 0)
    
#     # Store model in dictionary
#     models['isolation_forest'] = (if_model, features[features['if_is_anomaly'] == 1])
    
#     # 2. Local Outlier Factor for better local density-based detection
#     print("Training Local Outlier Factor model...")
#     lof = LocalOutlierFactor(
#         n_neighbors=20,
#         contamination=contamination,
#         novelty=False,
#         n_jobs=-1
#     )
    
#     # LOF doesn't have a dedicated predict method in training mode
#     features['lof_is_anomaly'] = (lof.fit_predict(X_scaled) == -1).astype(int)
    
#     # Store model in dictionary (though LOF in this mode isn't reusable for new data)
#     models['lof'] = (lof, features[features['lof_is_anomaly'] == 1])
    
#     # 3. One-Class SVM for robust anomaly detection
#     print("Training One-Class SVM model...")
#     ocsvm = OneClassSVM(
#         nu=contamination,
#         kernel='rbf',
#         gamma='scale'
#     )
    
#     ocsvm.fit(X_scaled)
    
#     features['ocsvm_anomaly'] = ocsvm.predict(X_scaled)
#     features['ocsvm_is_anomaly'] = features['ocsvm_anomaly'].apply(lambda x: 1 if x == -1 else 0)
    
#     # Store model in dictionary
#     models['ocsvm'] = (ocsvm, features[features['ocsvm_is_anomaly'] == 1])
    
#     # 4. DBSCAN with optimized parameters
#     print("Training DBSCAN model...")
#     # OPTICS is a more advanced version of DBSCAN that automatically determines proper eps values
#     dbscan = OPTICS(
#         min_samples=5,
#         metric='minkowski',
#         n_jobs=-1
#     )
    
#     features['dbscan_cluster'] = dbscan.fit_predict(X_scaled)
    
#     # Mark outliers (cluster -1) as anomalies
#     features['dbscan_is_anomaly'] = (features['dbscan_cluster'] == -1).astype(int)
    
#     # Store model in dictionary
#     models['dbscan'] = (dbscan, features[features['dbscan_is_anomaly'] == 1])
    
#     # Create ensemble anomaly detection with weighted voting
#     features['ensemble_score'] = (
#         features['if_is_anomaly'] * 0.35 + 
#         features['lof_is_anomaly'] * 0.2 + 
#         features['ocsvm_is_anomaly'] * 0.15 + 
#         features['dbscan_is_anomaly'] * 0.3
#     )
#     features['ensemble_is_anomaly'] = (features['ensemble_score'] >= 0.3).astype(int)
    
#     # Count anomalies for each method
#     if_anomaly_count = features['if_is_anomaly'].sum()
#     lof_anomaly_count = features['lof_is_anomaly'].sum()
#     ocsvm_anomaly_count = features['ocsvm_is_anomaly'].sum()
#     dbscan_anomaly_count = features['dbscan_is_anomaly'].sum()
#     ensemble_anomaly_count = features['ensemble_is_anomaly'].sum()
    
#     print(f"Isolation Forest: Detected {if_anomaly_count} anomalies out of {len(features)} intervals")
#     print(f"Local Outlier Factor: Detected {lof_anomaly_count} anomalies out of {len(features)} intervals")
#     print(f"One-Class SVM: Detected {ocsvm_anomaly_count} anomalies out of {len(features)} intervals")
#     print(f"OPTICS (DBSCAN): Detected {dbscan_anomaly_count} anomalies out of {len(features)} intervals")
#     print(f"Ensemble: Detected {ensemble_anomaly_count} anomalies out of {len(features)} intervals")
    
#     # Enhanced visualization with Seaborn
#     plt.figure(figsize=(20, 16))
    
#     # Isolation Forest
#     plt.subplot(3, 2, 1)
#     sns.scatterplot(x=features['initiated_time'], y=features['transaction_count'], 
#                    hue=features['if_is_anomaly'], palette=['blue', 'red'], alpha=0.7)
#     plt.title('Isolation Forest Anomaly Detection', fontsize=14)
#     plt.xlabel('Time', fontsize=12)
#     plt.ylabel('Transaction Count', fontsize=12)
#     plt.legend(title='Anomaly', labels=['Normal', 'Anomaly'])
    
#     # Local Outlier Factor
#     plt.subplot(3, 2, 2)
#     sns.scatterplot(x=features['initiated_time'], y=features['transaction_count'], 
#                    hue=features['lof_is_anomaly'], palette=['blue', 'red'], alpha=0.7)
#     plt.title('Local Outlier Factor Anomaly Detection', fontsize=14)
#     plt.xlabel('Time', fontsize=12)
#     plt.ylabel('Transaction Count', fontsize=12)
#     plt.legend(title='Anomaly', labels=['Normal', 'Anomaly'])
    
#     # One-Class SVM
#     plt.subplot(3, 2, 3)
#     sns.scatterplot(x=features['initiated_time'], y=features['transaction_count'], 
#                    hue=features['ocsvm_is_anomaly'], palette=['blue', 'red'], alpha=0.7)
#     plt.title('One-Class SVM Anomaly Detection', fontsize=14)
#     plt.xlabel('Time', fontsize=12)
#     plt.ylabel('Transaction Count', fontsize=12)
#     plt.legend(title='Anomaly', labels=['Normal', 'Anomaly'])
    
#     # DBSCAN
#     plt.subplot(3, 2, 4)
#     sns.scatterplot(x=features['initiated_time'], y=features['transaction_count'], 
#                    hue=features['dbscan_is_anomaly'], palette=['blue', 'red'], alpha=0.7)
#     plt.title('OPTICS (Advanced DBSCAN) Anomaly Detection', fontsize=14)
#     plt.xlabel('Time', fontsize=12)
#     plt.ylabel('Transaction Count', fontsize=12)
#     plt.legend(title='Anomaly', labels=['Normal', 'Anomaly'])
    
#     # Ensemble
#     plt.subplot(3, 2, 5)
#     sns.scatterplot(x=features['initiated_time'], y=features['transaction_count'], 
#                    hue=features['ensemble_is_anomaly'], palette=['blue', 'red'], alpha=0.7)
#     plt.title('Weighted Ensemble Anomaly Detection', fontsize=14)
#     plt.xlabel('Time', fontsize=12)
#     plt.ylabel('Transaction Count', fontsize=12)
#     plt.legend(title='Anomaly', labels=['Normal', 'Anomaly'])
    
#     # Z-Score visualization
#     if 'z_score' in features.columns:
#         plt.subplot(3, 2, 6)
#         sns.lineplot(x=features['initiated_time'], y=features['z_score'], color='purple')
#         plt.axhline(y=3, color='red', linestyle='--', label='3σ Threshold')
#         plt.axhline(y=-3, color='red', linestyle='--')
#         plt.title('Z-Score of Transaction Counts', fontsize=14)
#         plt.xlabel('Time', fontsize=12)
#         plt.ylabel('Z-Score', fontsize=12)
#         plt.legend()
    
#     plt.tight_layout()
#     plt.savefig('images/enhanced_traffic_anomalies_comparison.png', dpi=300)
#     plt.close()
    
#     # Create time series visualization of anomalies
#     plt.figure(figsize=(16, 8))
#     plt.plot(features['initiated_time'], features['transaction_count'], color='blue', alpha=0.5)
#     plt.scatter(features.loc[features['ensemble_is_anomaly'] == 1, 'initiated_time'],
#                features.loc[features['ensemble_is_anomaly'] == 1, 'transaction_count'],
#                color='red', label='Anomalies', s=50)
    
#     plt.title('Transaction Count Time Series with Detected Anomalies', fontsize=16)
#     plt.xlabel('Time', fontsize=14)
#     plt.ylabel('Transaction Count', fontsize=14)
#     plt.legend(fontsize=12)
#     plt.grid(True, alpha=0.3)
#     plt.tight_layout()
#     plt.savefig('images/transaction_timeseries_anomalies.png', dpi=300)
#     plt.close()
    
#     # Return models and anomalies
#     ensemble_anomalies = features[features['ensemble_is_anomaly'] == 1]
#     return models, scaler, ensemble_anomalies, features

# # Build traffic anomaly detector
# print("\nBuilding advanced traffic anomaly detection models...")
# anomaly_models, anomaly_scaler, anomalies, features_with_anomalies = build_traffic_anomaly_detector(ts_15min)

# # Show the detected anomalies
# print("\nTop 10 detected traffic anomalies:")
# print(anomalies[['initiated_time', 'transaction_count']].sort_values('transaction_count', ascending=False).head(10))

# # 3.2 Enhanced Toll Skipping Detection
# print("\n3.2 Enhanced Toll Skipping Detection")

# def detect_advanced_toll_skipping(df):

#     # Print columns for debugging
#     print("Available columns in dataframe:")
#     print(df.columns.tolist())
    
#     # Check if the vehicle registration column exists
#     vehicle_col = 'vehicle_regn_number'
    
#     if vehicle_col not in df.columns:
#         # Try to find alternatives
#         possible_columns = [col for col in df.columns if any(term in col.lower() for term in 
#                            ['vehicle', 'regn', 'registration', 'reg', 'tag', 'transponder', 'id'])]
        
#         if possible_columns:
#             vehicle_col = possible_columns[0]
#             print(f"Using alternative column: '{vehicle_col}'")
#         else:
#             print("No suitable vehicle identifier found. Creating synthetic ID.")
#             df['synthetic_vehicle_id'] = range(1, len(df) + 1)
#             vehicle_col = 'synthetic_vehicle_id'
    
#     print(f"Using '{vehicle_col}' as vehicle identifier")
    
#     # Create a copy of dataframe sorted by time for each vehicle
#     df_sorted = df.sort_values([vehicle_col, 'initiated_time']).copy()
    
#     # Create trip-level features
#     print("Creating advanced trip-level features...")
    
#     # Group transactions into trips with temporal proximity
#     df_sorted['prev_time'] = df_sorted.groupby(vehicle_col)['initiated_time'].shift(1)
#     df_sorted['time_since_prev'] = (df_sorted['initiated_time'] - df_sorted['prev_time']).dt.total_seconds() / 60
    
#     # Identify new trips (more than 120 minutes since last transaction)
#     df_sorted['new_trip'] = (df_sorted['time_since_prev'].isna() | (df_sorted['time_since_prev'] > 120)).astype(int)
    
#     # Assign trip IDs
#     df_sorted['trip_id'] = df_sorted.groupby(vehicle_col)['new_trip'].cumsum()
#     df_sorted['trip_key'] = df_sorted[vehicle_col].astype(str) + '_' + df_sorted['trip_id'].astype(str)
    
#     # Group by vehicle identifier
#     print(f"Grouping by {vehicle_col}...")
#     vehicle_stats = df_sorted.groupby(vehicle_col).agg({
#         'initiated_time': ['min', 'max', 'count'],
#         'merchant_name': 'nunique',
#         'txn_amount': ['sum', 'mean', 'std'],
#         'trip_id': 'max'  # Number of trips
#     })
    
#     # Flatten multi-level columns
#     vehicle_stats.columns = ['_'.join(col).strip('_') for col in vehicle_stats.columns.values]
    
#     # Rename columns for clarity
#     vehicle_stats = vehicle_stats.rename(columns={
#         'initiated_time_min': 'first_seen',
#         'initiated_time_max': 'last_seen',
#         'initiated_time_count': 'total_transactions',
#         'merchant_name_nunique': 'unique_plazas',
#         'txn_amount_sum': 'total_amount',
#         'txn_amount_mean': 'avg_amount_per_transaction',
#         'txn_amount_std': 'amount_std',
#         'trip_id_max': 'total_trips'
#     })
    
#     # Calculate additional features
#     vehicle_stats['time_span_hours'] = (vehicle_stats['last_seen'] - vehicle_stats['first_seen']).dt.total_seconds() / 3600
#     vehicle_stats['trips_per_hour'] = vehicle_stats['total_trips'] / vehicle_stats['time_span_hours'].clip(lower=1)
#     vehicle_stats['transactions_per_trip'] = vehicle_stats['total_transactions'] / vehicle_stats['total_trips']
#     vehicle_stats['plaza_to_trip_ratio'] = vehicle_stats['unique_plazas'] / vehicle_stats['total_trips']
#     vehicle_stats['transactions_per_plaza'] = vehicle_stats['total_transactions'] / vehicle_stats['unique_plazas'].clip(lower=1)
    
#     # Analyze trip level behavior
#     trip_stats = df_sorted.groupby(['trip_key', vehicle_col]).agg({
#         'merchant_name': 'nunique',
#         'txn_amount': ['sum', 'count'],
#         'initiated_time': ['min', 'max']
#     })
    
#     # Flatten multi-level columns
#     trip_stats.columns = ['_'.join(col).strip('_') for col in trip_stats.columns.values]
    
#     # Calculate trip duration
#     trip_stats['trip_duration_min'] = (trip_stats['initiated_time_max'] - trip_stats['initiated_time_min']).dt.total_seconds() / 60
    
#     # Aggregate trip stats by vehicle
#     trip_aggregates = trip_stats.groupby(level=1).agg({
#         'merchant_name_nunique': ['mean', 'std', 'max'],
#         'txn_amount_sum': ['mean', 'std'],
#         'txn_amount_count': ['mean', 'std', 'max'],
#         'trip_duration_min': ['mean', 'std', 'max']
#     })
    
#     # Flatten multi-level columns again
#     trip_aggregates.columns = ['_'.join(col).strip('_') for col in trip_aggregates.columns.values]
    
#     # Merge with vehicle stats
#     vehicle_features = vehicle_stats.join(trip_aggregates)
    
#     # Replace NaN with 0 (for std deviations of vehicles with single trips)
#     vehicle_features = vehicle_features.fillna(0)
    
#     # Filter for vehicles with multiple trips
#     multi_trip_vehicles = vehicle_features[vehicle_features['total_trips'] > 1].copy()
    
#     models = {}
    
#     if len(multi_trip_vehicles) > 0:
#         print(f"Found {len(multi_trip_vehicles)} vehicles with multiple trips")
        
#         # PCA for dimensionality reduction and feature engineering
#         print("Performing PCA for dimensionality reduction...")
        
#         # Select only numeric columns for analysis
#         numeric_cols = multi_trip_vehicles.select_dtypes(include=np.number).columns.tolist()
        
#         # Feature selection to avoid curse of dimensionality
#         feature_cols = [col for col in numeric_cols if ('time_' not in col and 'seen' not in col)]
        
#         # Scale features with RobustScaler to handle outliers better
#         scaler = RobustScaler()
#         features_scaled = scaler.fit_transform(multi_trip_vehicles[feature_cols])
        
#         joblib.dump(scaler, 'models/toll_scaler.pkl')
#         # Perform PCA
#         pca = PCA(n_components=min(len(feature_cols), len(multi_trip_vehicles)-1))
#         pca_result = pca.fit_transform(features_scaled)
        
#         # Determine optimal number of components (explain 90% of variance)
#         var_explained = pca.explained_variance_ratio_.cumsum()
#         n_components = np.argmax(var_explained >= 0.9) + 1
        
#         # Ensure minimum 2 components for visualization
#         n_components = max(2, n_components)
#         print(f"Using {n_components} principal components (explaining {var_explained[n_components-1]*100:.2f}% of variance)")
        
#         # Reduce dimensions
#         pca = PCA(n_components=n_components)
#         pca_result = pca.fit_transform(features_scaled)
        
#         joblib.dump(pca, 'models/toll_pca.pkl')
#         # Create DataFrame with PCA results
#         pca_df = pd.DataFrame(
#             data=pca_result,
#             columns=[f'PC{i+1}' for i in range(n_components)],
#             index=multi_trip_vehicles.index
#         )
        
#         # Add back an important original feature for interpretability
#         pca_df['plaza_to_trip_ratio'] = multi_trip_vehicles['plaza_to_trip_ratio']
        
#         # Try multiple anomaly detection methods
        
#         # 1. Isolation Forest for outlier detection
#         print("\nPerforming Isolation Forest outlier detection...")
#         iso_features = pca_df[[f'PC{i+1}' for i in range(n_components)] + ['plaza_to_trip_ratio']]

#         iso_forest = IsolationForest(
#             n_estimators=200,
#             contamination=0.05,
#             bootstrap=True,
#             n_jobs=-1,
#             random_state=42
#         )

#         # Store results in NEW columns, don't modify existing features
#         pca_df['if_anomaly'] = iso_forest.fit_predict(iso_features)
#         pca_df['if_is_anomaly'] = (pca_df['if_anomaly'] == -1).astype(int)
#         pca_df['if_score'] = iso_forest.score_samples(iso_features)
        
#         # Store model
#         models['isolation_forest'] = iso_forest
#         joblib.dump(iso_forest, 'models/toll_isolation_forest.pkl')
#         # 2. One-Class SVM
#         print("Performing One-Class SVM outlier detection...")
#         ocsvm = OneClassSVM(
#             nu=0.05,
#             kernel='rbf',
#             gamma='scale'
#         )
        
#         # Corrected code - use only the original PCA components and ratio
#         ocsvm_features = pca_df[[f'PC{i+1}' for i in range(n_components)] + ['plaza_to_trip_ratio']]
#         pca_df['ocsvm_anomaly'] = ocsvm.fit_predict(ocsvm_features)
#         pca_df['ocsvm_is_anomaly'] = (pca_df['ocsvm_anomaly'] == -1).astype(int)
        
#         # Store model
#         models['ocsvm'] = ocsvm
        
#         joblib.dump(ocsvm, 'models/toll_oneclass_svm.pkl')
#         # 3. K-Means clustering
#         print("Performing K-Means clustering...")
        
#         # Determine optimal number of clusters using silhouette score or elbow method
#         from sklearn.metrics import silhouette_score
        
#         max_clusters = min(10, len(pca_df) - 1)
#         if max_clusters < 2:
#             max_clusters = 2  # Need at least 2 clusters
            
#         silhouette_scores = []
#         inertias = []
        
#         for k in range(2, max_clusters + 1):
#             kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
#             labels = kmeans.fit_predict(pca_df[[f'PC{i+1}' for i in range(n_components)] + ['plaza_to_trip_ratio']])
#             silhouette_scores.append(silhouette_score(
#                 pca_df[[f'PC{i+1}' for i in range(n_components)] + ['plaza_to_trip_ratio']], 
#                 labels, 
#                 metric='euclidean'
#             ))
#             inertias.append(kmeans.inertia_)
        
#         # Find optimal k (highest silhouette score)
#         optimal_k = np.argmax(silhouette_scores) + 2  # +2 because range starts at 2
        
#         print(f"Optimal number of clusters determined: {optimal_k}")
        
#         # Perform KMeans clustering with optimal k
#         kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
#         kmeans_features = pca_df[[f'PC{i+1}' for i in range(n_components)] + ['plaza_to_trip_ratio']]
#         pca_df['kmeans_cluster'] = kmeans.fit_predict(kmeans_features)
        
#         # Store model
#         models['kmeans'] = kmeans
        
#         # Visualize the clusters and anomalies in 2D PCA space
#         plt.figure(figsize=(16, 12))
        
#         # Plot K-Means clusters
#         plt.subplot(2, 2, 1)
#         if n_components >= 2:
#             scatter = plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['kmeans_cluster'], 
#                                 cmap='viridis', alpha=0.7, s=50)
#             plt.ylabel('Principal Component 2', fontsize=12)
#         else:
#             scatter = plt.scatter(pca_df['PC1'], pca_df['plaza_to_trip_ratio'], 
#                                 c=pca_df['kmeans_cluster'], cmap='viridis', alpha=0.7, s=50)
#             plt.ylabel('Plaza/Trip Ratio', fontsize=12)
#         plt.colorbar(scatter, label='Cluster')
#         plt.title('K-Means Clustering', fontsize=14)
#         plt.xlabel('Principal Component 1', fontsize=12)
#         plt.grid(True, alpha=0.3)
        
#         # Plot Isolation Forest anomalies
#         plt.subplot(2, 2, 2)
#         if n_components >= 2:
#             plt.scatter(pca_df.loc[pca_df['if_is_anomaly'] == 0, 'PC1'], 
#                         pca_df.loc[pca_df['if_is_anomaly'] == 0, 'PC2'], 
#                         c='blue', label='Normal', alpha=0.5)
#             plt.scatter(pca_df.loc[pca_df['if_is_anomaly'] == 1, 'PC1'], 
#                         pca_df.loc[pca_df['if_is_anomaly'] == 1, 'PC2'], 
#                         c='red', label='Anomaly', alpha=0.7, s=70)
#             plt.ylabel('Principal Component 2', fontsize=12)
#         else:
#             plt.scatter(pca_df.loc[pca_df['if_is_anomaly'] == 0, 'PC1'], 
#                         pca_df.loc[pca_df['if_is_anomaly'] == 0, 'plaza_to_trip_ratio'], 
#                         c='blue', label='Normal', alpha=0.5)
#             plt.scatter(pca_df.loc[pca_df['if_is_anomaly'] == 1, 'PC1'], 
#                         pca_df.loc[pca_df['if_is_anomaly'] == 1, 'plaza_to_trip_ratio'], 
#                         c='red', label='Anomaly', alpha=0.7, s=70)
#             plt.ylabel('Plaza/Trip Ratio', fontsize=12)
#         plt.title('Isolation Forest Anomalies', fontsize=14)
#         plt.xlabel('Principal Component 1', fontsize=12)
#         plt.legend()
#         plt.grid(True, alpha=0.3)
        
#         # Plot One-Class SVM anomalies
#         plt.subplot(2, 2, 3)
#         if n_components >= 2:
#             plt.scatter(pca_df.loc[pca_df['ocsvm_is_anomaly'] == 0, 'PC1'], 
#                         pca_df.loc[pca_df['ocsvm_is_anomaly'] == 0, 'PC2'], 
#                         c='blue', label='Normal', alpha=0.5)
#             plt.scatter(pca_df.loc[pca_df['ocsvm_is_anomaly'] == 1, 'PC1'], 
#                         pca_df.loc[pca_df['ocsvm_is_anomaly'] == 1, 'PC2'], 
#                         c='red', label='Anomaly', alpha=0.7, s=70)
#             plt.ylabel('Principal Component 2', fontsize=12)
#         else:
#             plt.scatter(pca_df.loc[pca_df['ocsvm_is_anomaly'] == 0, 'PC1'], 
#                         pca_df.loc[pca_df['ocsvm_is_anomaly'] == 0, 'plaza_to_trip_ratio'], 
#                         c='blue', label='Normal', alpha=0.5)
#             plt.scatter(pca_df.loc[pca_df['ocsvm_is_anomaly'] == 1, 'PC1'], 
#                         pca_df.loc[pca_df['ocsvm_is_anomaly'] == 1, 'plaza_to_trip_ratio'], 
#                         c='red', label='Anomaly', alpha=0.7, s=70)
#             plt.ylabel('Plaza/Trip Ratio', fontsize=12)
#         plt.title('One-Class SVM Anomalies', fontsize=14)
#         plt.xlabel('Principal Component 1', fontsize=12)
#         plt.legend()
#         plt.grid(True, alpha=0.3)
        
#         # Plot combined anomaly score
#         plt.subplot(2, 2, 4)
        
#         # Create ensemble score
#         pca_df['ensemble_score'] = pca_df['if_is_anomaly'] + pca_df['ocsvm_is_anomaly']
#         pca_df['ensemble_is_anomaly'] = (pca_df['ensemble_score'] > 0).astype(int)
        
#         if n_components >= 2:
#             plt.scatter(pca_df.loc[pca_df['ensemble_is_anomaly'] == 0, 'PC1'], 
#                         pca_df.loc[pca_df['ensemble_is_anomaly'] == 0, 'PC2'], 
#                         c='blue', label='Normal', alpha=0.5)
#             plt.scatter(pca_df.loc[pca_df['ensemble_is_anomaly'] == 1, 'PC1'], 
#                         pca_df.loc[pca_df['ensemble_is_anomaly'] == 1, 'PC2'], 
#                         c='red', label='Anomaly', alpha=0.7, s=70)
#             plt.ylabel('Principal Component 2', fontsize=12)
#         else:
#             plt.scatter(pca_df.loc[pca_df['ensemble_is_anomaly'] == 0, 'PC1'], 
#                         pca_df.loc[pca_df['ensemble_is_anomaly'] == 0, 'plaza_to_trip_ratio'], 
#                         c='blue', label='Normal', alpha=0.5)
#             plt.scatter(pca_df.loc[pca_df['ensemble_is_anomaly'] == 1, 'PC1'], 
#                         pca_df.loc[pca_df['ensemble_is_anomaly'] == 1, 'plaza_to_trip_ratio'], 
#                         c='red', label='Anomaly', alpha=0.7, s=70)
#             plt.ylabel('Plaza/Trip Ratio', fontsize=12)
#         plt.title('Ensemble Anomaly Detection', fontsize=14)
#         plt.xlabel('Principal Component 1', fontsize=12)
#         plt.legend()
#         plt.grid(True, alpha=0.3)
        
#         plt.tight_layout()
#         plt.savefig('images/advanced_toll_skipping_analysis.png', dpi=300)
#         plt.close()
        
#         # Analyze clusters to identify potential toll skipping patterns
        
#         # Merge cluster information back to original features for analysis
#         cluster_map = pd.DataFrame({
#             'kmeans_cluster': pca_df['kmeans_cluster'],
#             'if_is_anomaly': pca_df['if_is_anomaly'],
#             'ocsvm_is_anomaly': pca_df['ocsvm_is_anomaly'],
#             'ensemble_is_anomaly': pca_df['ensemble_is_anomaly']
#         })
        
#         multi_trip_vehicles = multi_trip_vehicles.join(cluster_map).reset_index()
        
#         # Analyze each cluster
#         cluster_stats = multi_trip_vehicles.groupby('kmeans_cluster').agg({
#             'plaza_to_trip_ratio': 'mean',
#             'transactions_per_trip': 'mean',
#             'trips_per_hour': 'mean',
#             'total_trips': 'mean',
#             'if_is_anomaly': 'mean',  # Proportion of anomalies per cluster
#             vehicle_col: 'count'  # Now works because vehicle_col is a column
#         }).rename(columns={vehicle_col: 'vehicle_count'})
        
#         print("\nCluster statistics:")
#         print(cluster_stats)
        
#         # Identify potential toll skipping clusters
#         # Lower plaza_to_trip_ratio suggests toll skipping (fewer tolls per trip)
#         toll_skip_score = cluster_stats['plaza_to_trip_ratio'] * 0.7 + \
#                         cluster_stats['if_is_anomaly'] * 0.3
#         potential_skip_cluster = toll_skip_score.idxmin()
        
#         print(f"\nPotential toll skipping cluster identified: Cluster {potential_skip_cluster}")
#         print("Cluster characteristics:")
#         print(cluster_stats.loc[potential_skip_cluster])
        
#         # Get vehicles in the potential toll skipping cluster
#         cluster_skip_vehicles = multi_trip_vehicles[multi_trip_vehicles['kmeans_cluster'] == potential_skip_cluster]
        
#         # Combine with ensemble anomalies from PCA analysis
#         ensemble_skip_vehicles = multi_trip_vehicles[multi_trip_vehicles['ensemble_is_anomaly'] == 1]
        
#         # Combine results from both cluster analysis and ensemble anomalies
#         combined_suspects = pd.concat([cluster_skip_vehicles, ensemble_skip_vehicles]).drop_duplicates()
#         print(f"\nTotal unique suspect vehicles identified: {len(combined_suspects)}")
        
#         # Advanced filtering using supervised learning
#         print("\nApplying supervised anomaly scoring...")
        
#         # Create features for supervised model
#         supervised_features = combined_suspects[feature_cols].copy()
        
#         # Train XGBoost classifier on cluster labels (pseudo-labels)
#         xgb_model = xgb.XGBClassifier(
#             n_estimators=150,
#             max_depth=5,
#             learning_rate=0.1,
#             subsample=0.8,
#             colsample_bytree=0.8,
#             random_state=42
#         )
        
#         # Use cluster labels as pseudo-targets (1 for suspicious cluster, 0 others)
#         pseudo_targets = (combined_suspects['kmeans_cluster'] == potential_skip_cluster).astype(int)
        
#         # Train-test split
#         X_train, X_test, y_train, y_test = train_test_split(
#             supervised_features, pseudo_targets, test_size=0.2, random_state=42
#         )
        
#         # Train model
#         xgb_model.fit(X_train, y_train)
#         joblib.dump(xgb_model, 'models/toll_xgboost.pkl')

#         # Predict probabilities
#         combined_suspects['fraud_probability'] = xgb_model.predict_proba(supervised_features)[:, 1]
        
#         # Filter high-probability suspects
#         high_prob_suspects = combined_suspects[combined_suspects['fraud_probability'] > 0.7]
#         print(f"High probability toll skipping suspects: {len(high_prob_suspects)}")
        
#         # Feature importance analysis
#         plt.figure(figsize=(12, 8))
#         xgb.plot_importance(xgb_model, max_num_features=10, importance_type='weight')
#         plt.title('XGBoost Feature Importance for Toll Skipping Detection')
#         plt.tight_layout()
#         plt.savefig('images/toll_skip_feature_importance.png', dpi=300)
#         plt.close()
        
#         # Temporal pattern analysis of suspects
#         suspect_transactions = df_sorted[df_sorted[vehicle_col].isin(high_prob_suspects.index)]
        
#         plt.figure(figsize=(16, 8))
#         plt.hist(suspect_transactions['initiated_time'].dt.hour, bins=24, 
#                 alpha=0.7, label='Suspects', color='red')
#         plt.hist(df_sorted['initiated_time'].dt.hour, bins=24, 
#                 alpha=0.3, label='All Vehicles', color='blue')
#         plt.title('Temporal Distribution of Suspect Transactions vs Overall')
#         plt.xlabel('Hour of Day')
#         plt.ylabel('Transaction Count')
#         plt.legend()
#         plt.grid(True, alpha=0.3)
#         plt.tight_layout()
#         plt.savefig('images/toll_skip_temporal_patterns.png', dpi=300)
#         plt.close()
        
#         feature_cols = [
#         'total_trips',
#         'unique_plazas',
#         'trips_per_hour',
#         'avg_amount_per_transaction',
#         'plaza_to_trip_ratio',
#         'trip_duration_min_mean',
#         'transactions_per_trip',
#         'amount_std',
#         'total_transactions',
#         'merchant_name_nunique_mean',
#         'txn_amount_sum_std'
#     ]
    
#         # Filter available features
#         feature_cols = [col for col in feature_cols if col in numeric_cols]
        
#         # [Rest of the function remains the same until return]
        
#         # Save feature columns for inference
#         joblib.dump(feature_cols, 'models/toll_feature_columns.pkl')
        
#         return high_prob_suspects, models, scaler
#     else:
#         print("No multi-trip vehicles available for analysis")
#         return None, None, None

# # Detect advanced toll skipping patterns
# print("\nDetecting advanced toll skipping patterns...")
# skip_vehicles_enhanced, skip_models_enhanced, skip_scaler_enhanced = detect_advanced_toll_skipping(df)

# if skip_vehicles_enhanced is not None and len(skip_vehicles_enhanced) > 0:
#     print("\nTop 5 high-probability toll skipping suspects:")
#     print(skip_vehicles_enhanced.sort_values('fraud_probability', ascending=False).head())
# else:
#     print("No toll skipping suspects detected")

# # 3.3 Real-time Anomaly Detection Pipeline
# print("\n3.3 Real-time Anomaly Detection Pipeline")

# class RealTimeAnomalyDetector:
#     """Class for real-time anomaly detection with model persistence"""
    
#     def __init__(self, traffic_model, toll_skip_model, traffic_scaler, toll_skip_scaler):
#         self.traffic_model = traffic_model
#         self.toll_skip_model = toll_skip_model
#         self.traffic_scaler = traffic_scaler
#         self.toll_skip_scaler = toll_skip_scaler
#         self.anomaly_log = pd.DataFrame(columns=['timestamp', 'type', 'severity', 'details'])
        
#     def log_anomaly(self, anomaly_type, severity, details):
#         """Log detected anomalies with metadata"""
#         new_entry = pd.DataFrame([{
#             'timestamp': datetime.now(),
#             'type': anomaly_type,
#             'severity': severity,
#             'details': str(details)
#         }])
#         self.anomaly_log = pd.concat([self.anomaly_log, new_entry], ignore_index=True)
        
#     def detect_traffic_anomalies(self, new_data):
#         """Detect traffic pattern anomalies in real-time"""
#         # Feature engineering
#         features = self._create_traffic_features(new_data)
        
#         # Scale features
#         scaled_data = self.traffic_scaler.transform(features)
        
#         # Get ensemble prediction
#         if_pred = self.traffic_model['isolation_forest'][0].predict(scaled_data)
#         lof_pred = self.traffic_model['lof'][0].fit_predict(scaled_data)
#         ocsvm_pred = self.traffic_model['ocsvm'][0].predict(scaled_data)
#         dbscan_pred = self.traffic_model['dbscan'][0].fit_predict(scaled_data)
        
#         # Ensemble scoring
#         ensemble_score = (np.where(if_pred == -1, 0.35, 0) +
#                          np.where(lof_pred == -1, 0.2, 0) +
#                          np.where(ocsvm_pred == -1, 0.15, 0) +
#                          np.where(dbscan_pred == -1, 0.3, 0))
        
#         anomalies = ensemble_score >= 0.3
#         return anomalies, ensemble_score
    
#     def detect_toll_skipping(self, vehicle_data):
#         """Detect potential toll skipping in real-time"""
#         # Feature engineering
#         feature_cols = joblib.load('models/toll_feature_columns.pkl')
        
#         features = self._create_vehicle_features(vehicle_data)
        
#         features = features[feature_cols]
#         # Scale features
#         scaled_data = self.toll_skip_scaler.transform(features)
        
#         # Dimensionality reduction
#         pca_data = self.toll_skip_model['pca'].transform(scaled_data)
        
#         # Predict using ensemble model
#         if_pred = self.toll_skip_model['isolation_forest'].predict(pca_data)
#         ocsvm_pred = self.toll_skip_model['ocsvm'].predict(pca_data)
        
#         # Calculate probability
#         prob = (np.mean([if_pred == -1, ocsvm_pred == -1]) * 0.5 +
#                self.toll_skip_model['xgb'].predict_proba(features)[:, 1] * 0.5)
        
#         return prob
    
#     def _create_traffic_features(self, data):
#         """Create traffic features matching training format"""
#         features = data.copy()
#         features['hour'] = features['timestamp'].dt.hour
#         features['day_of_week'] = features['timestamp'].dt.dayofweek
#         features['is_weekend'] = features['day_of_week'].isin([5, 6]).astype(int)
#         features['month'] = features['timestamp'].dt.month
#         features['day'] = features['timestamp'].dt.day
#         return features[['transaction_count', 'hour', 'day_of_week', 'is_weekend', 'month', 'day']]
    
#     def _create_vehicle_features(self, data):
#         """Create vehicle features matching training format"""
#         features = data.copy()
#         features['time_span_hours'] = (features['last_seen'] - features['first_seen']).dt.total_seconds() / 3600
#         features['trips_per_hour'] = features['total_trips'] / features['time_span_hours'].clip(lower=1)
#         features['transactions_per_trip'] = features['total_transactions'] / features['total_trips']
#         features['plaza_to_trip_ratio'] = features['unique_plazas'] / features['total_trips']
#         return features[feature_cols]  # Use same features as training
    
#     def save_models(self, path='models'):
#         """Save all models and scalers to disk"""
#         joblib.dump(self.traffic_model, f'{path}/traffic_model.pkl')
#         joblib.dump(self.toll_skip_model, f'{path}/toll_skip_model.pkl')
#         joblib.dump(self.traffic_scaler, f'{path}/traffic_scaler.pkl')
#         joblib.dump(self.toll_skip_scaler, f'{path}/toll_skip_scaler.pkl')
#         print(f"Models saved to {path}")
        
#     @classmethod
#     def load_models(cls, path='models'):
#         """Load models from disk"""
#         return cls(
#             traffic_model=joblib.load(f'{path}/traffic_model.pkl'),
#             toll_skip_model=joblib.load(f'{path}/toll_skip_model.pkl'),
#             traffic_scaler=joblib.load(f'{path}/traffic_scaler.pkl'),
#             toll_skip_scaler=joblib.load(f'{path}/toll_skip_scaler.pkl')
#         )

# # Initialize real-time detector
# print("\nInitializing real-time anomaly detection system...")
# real_time_detector = RealTimeAnomalyDetector(
#     traffic_model=anomaly_models,
#     toll_skip_model=skip_models_enhanced,
#     traffic_scaler=anomaly_scaler,
#     toll_skip_scaler=skip_scaler_enhanced
# )

# # Save models for production use
# real_time_detector.save_models()

# print("\nEnhanced Anomaly Detection System implementation completed.")


3. ENHANCED ANOMALY DETECTION SYSTEM
--------------------------------------------------

3.1 Advanced Traffic Pattern Anomaly Detection

Building advanced traffic anomaly detection models...

Training Isolation Forest model...
Training Local Outlier Factor model...
Training One-Class SVM model...
Training DBSCAN model...
Isolation Forest: Detected 5 anomalies out of 96 intervals
Local Outlier Factor: Detected 5 anomalies out of 96 intervals
One-Class SVM: Detected 8 anomalies out of 96 intervals
OPTICS (DBSCAN): Detected 54 anomalies out of 96 intervals
Ensemble: Detected 55 anomalies out of 96 intervals

Top 10 detected traffic anomalies:
        initiated_time  transaction_count
73 2024-03-19 18:15:00               5598
61 2024-03-19 15:15:00               5582
46 2024-03-19 11:30:00               5185
45 2024-03-19 11:15:00               5135
75 2024-03-19 18:45:00               5128
63 2024-03-19 15:45:00               5078
47 2024-03-19 11:45:00               4978
62 2024-03-19 1

DEBUG: locator: <matplotlib.ticker.AutoLocator object at 0x32982ce80>



Cluster statistics:
                plaza_to_trip_ratio  transactions_per_trip  trips_per_hour  \
kmeans_cluster                                                               
0                          0.630067               1.071499        0.344079   
1                          0.758854               1.634447        0.264276   

                total_trips  if_is_anomaly  vehicle_count  
kmeans_cluster                                             
0                  2.113697       0.016219          49201  
1                  2.337620       0.437456           4277  

Potential toll skipping cluster identified: Cluster 0
Cluster characteristics:
plaza_to_trip_ratio          0.630067
transactions_per_trip        1.071499
trips_per_hour               0.344079
total_trips                  2.113697
if_is_anomaly                0.016219
vehicle_count            49201.000000
Name: 0, dtype: float64

Total unique suspect vehicles identified: 51236

Applying supervised anomaly scoring...
High 

<Figure size 1200x800 with 0 Axes>

In [17]:
# 4. SAVE MODELS AND RESULTS
import joblib
print("\n4. SAVING MODELS AND RESULTS")
print("-" * 50)

# Save traffic prediction model
if 'traffic_model' in locals():
    joblib.dump(traffic_model, 'models/traffic_prediction_model.pkl')
    print("Traffic prediction model saved as 'traffic_prediction_model.pkl'")

# Save anomaly detection models
if 'anomaly_models' in locals():
    joblib.dump((anomaly_models, anomaly_scaler), 'models/traffic_anomaly_models.pkl')
    print("Traffic anomaly detection models saved as 'traffic_anomaly_models.pkl'")

# Save vehicle class prediction model if available
if 'vc_model' in locals():
    joblib.dump(vc_model, 'models/vehicle_class_prediction_model.pkl')
    print("Vehicle class prediction model saved as 'vehicle_class_prediction_model.pkl'")

# Save toll skipping detection models if available
if 'skip_models' in locals() and skip_models is not None:
    joblib.dump((skip_models, skip_scaler), 'models/toll_skipping_models.pkl')
    print("Toll skipping detection models saved as 'toll_skipping_models.pkl'")

# Save vehicle behavior classifier if available
if 'vehicle_classifier' in locals():
    joblib.dump(vehicle_classifier, 'models/vehicle_behavior_classifier.pkl')
    print("Vehicle behavior classifier saved as 'vehicle_behavior_classifier.pkl'")

# Save the anomalies dataframe
if 'anomalies' in locals() and len(anomalies) > 0:
    anomalies.to_csv('data/detected_anomalies.csv', index=False)
    print("Detected anomalies saved as 'detected_anomalies.csv'")

# Save potential toll skipping vehicles
if 'potential_skip_vehicles' in locals() and potential_skip_vehicles is not None:
    potential_skip_vehicles.to_csv('data/potential_toll_skipping_vehicles.csv')
    print("Potential toll skipping vehicles saved as 'potential_toll_skipping_vehicles.csv'")

# Save traffic pattern analysis results
if 'hourly_patterns' in locals() and 'daily_patterns' in locals():
    hourly_patterns.to_csv('data/hourly_traffic_patterns.csv', index=False)
    daily_patterns.to_csv('data/daily_traffic_patterns.csv', index=False)
    print("Traffic pattern analysis results saved as CSV files")

print("\nPhase 2: Enhanced Analytics Models complete!")


4. SAVING MODELS AND RESULTS
--------------------------------------------------
Traffic prediction model saved as 'traffic_prediction_model.pkl'
Traffic anomaly detection models saved as 'traffic_anomaly_models.pkl'
Vehicle class prediction model saved as 'vehicle_class_prediction_model.pkl'
Detected anomalies saved as 'detected_anomalies.csv'

Phase 2: Enhanced Analytics Models complete!
